# Data Preprocessing

1. boxoffice
2. detail
3. rating
4. data table
5. search result  
> _search result was done in Crawler.ipynb_
6. input movie_rating for missing rating

In [1]:
import pandas as pd
import datetime
import requests
import json
import csv
# import matplotlib as plt

#-*- coding: utf-8 -*-

# kim key no.1 : c15fcbaab76cc525e9342c5e4dda712f
# kim key no.2 : c3268b0918814ae0a2bc298f7ba893a3
# lee key no.1 : 77a87d41d6e2cbe3131a190d5c48514e
# lee key no.2 : e53650978fc97b47c7f932136ce2c08a

## Crawling movie boxoffice data



In [117]:
key1 = 'e53650978fc97b47c7f932136ce2c08a'

def crawl_movie_boxoffice_data(start_date, end_date, key = key1) :
    result_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
    
    for single_date in pd.date_range(start_date, end_date) :
        for multi, rep in zip(['Y', 'Y', 'N', 'N'], ['K', 'F', 'K', 'F']) :
            payload = {
                'key' : key,
                'targetDt' : single_date.strftime('%Y%m%d'),
                'itemPerPage' : '10',
                'multiMovieYn' : multi,
                'repNationCd' : rep
            }
            
            req = requests.get(url, params = payload)

            for item in req.json()['boxOfficeResult']['dailyBoxOfficeList'] :
                temp_list = []
                header_list = []
                for header, value in item.items() :
                    header_list.append(header)
                    temp_list.append(value)
                    
                temp_list.append(single_date)
                header_list.append('CurrentDate')
                
                temp_list.append(multi)
                header_list.append('multi')
                
                temp_list.append(rep)
                header_list.append('Nation')
                
                result_list.append(temp_list)
                
    return pd.DataFrame(result_list, columns = header_list)

movie_boxoffice_list = crawl_movie_boxoffice_data('20020101', '20021231', key = key1)
movie_boxoffice_list.to_csv('movie_boxoffice_2002.csv', index = False)

UnboundLocalError: local variable 'header_list' referenced before assignment

In [25]:
## concatenate boxoffice files
# 2013 ~ 2018

all_filenames = []
for i in range(2013, 2019) :
    all_filenames.append('movie_boxoffice_'+str(i)+'.csv')
movie_boxoffice_combined = pd.concat([pd.read_csv(f) for f in all_filenames ])

movie_boxoffice_combined.to_csv('movie_boxoffice.csv', index = False, encoding = 'utf-8')

In [19]:
## test

movie_boxoffice_file_2017 = open('movie_boxoffice_2017.csv', 'r', encoding = 'utf-8')
movie_boxoffice_file_2018 = open('movie_boxoffice_2018.csv', 'r', encoding = 'utf-8')

fa=pd.read_csv(movie_boxoffice_file_2017)
fb=pd.read_csv(movie_boxoffice_file_2018)

print(len(fa))
print(len(fb))
with open('movie_boxoffice.csv', 'r', encoding = 'utf-8') as infile :
    fc=pd.read_csv(infile)
    print(len(fc))

14446
14560
29006


In [26]:
movie_boxoffice_list = pd.read_csv('movie_boxoffice.csv')

len(movie_boxoffice_list)

86747

## Crawling movie rating

from Naver movie

In [27]:
# movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_f.movieCd.drop_duplicates(), 'rating' : 0})
movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_list.movieCd.drop_duplicates(), 'rating' : 0})

len(movie_rating_list)

6195

In [28]:
movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

In [45]:
## sorting movie_rating_list
# and divide 3 parts to use seperate keys

movie_rating_list=movie_rating_list.sort_values(by='movieCd', ascending=True).reset_index()
mrl1 = movie_rating_list.iloc[0:3000]
mrl2 = movie_rating_list.iloc[3000:6000]
mrl3 = movie_rating_list.iloc[6000:]



3000
3000
195


In [50]:
key2 = '77a87d41d6e2cbe3131a190d5c48514e'

def crawl_movie_detail_data(lists, key = key2) :
    movie_detail_json = dict()
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    
    for movieCd in lists['movieCd'] :
        payload = {
            'key' : key,
            'movieCd' : movieCd
        }
        req = requests.get(url, params = payload)
        
        try :
            movie_detail_json[movieCd] = req.json()['movieInfoResult']['movieInfo']
        except :
            print("Error !")
            print(req.json())
        
    return movie_detail_json

movie_detail = crawl_movie_detail_data(mrl3)

with open('movie_detail_3.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

In [11]:
with open('movie_detail_1.json', 'r') as f :
    movie_detail_1 = json.load(f)
with open('movie_detail_2.json', 'r') as f :
    movie_detail_2 = json.load(f)
with open('movie_detail_3.json', 'r') as f :
    movie_detail_3 = json.load(f)

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [30]:
## concatenate movie detail(json type) files
# 2013 ~ 2018

movie_detail = dict()
for j in movie_detail_1 :
    movie_detail[j] = movie_detail_1[j]

for j in movie_detail_2 :
    movie_detail[j] = movie_detail_2[j]

for j in movie_detail_3 :
    movie_detail[j] = movie_detail_3[j]

with open('movie_detail.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

3000
6000
6195


In [51]:
movie_rating_list = pd.read_csv('movie_rating_list.csv')

movie_rating_list = movie_rating_list.sort_values(by='movieCd', ascending = True)

movie_rating_list = movie_rating_list.drop(columns=['index'])

movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

In [57]:
movie_rating_list = pd.read_csv('movie_rating_list.csv')

movie_detail[movie_rating_list.iloc[i,0]]['movieNm']

'오렌지필름 기획전 - do not reply'

In [247]:
## Crawling user's movie rating
# from Naver api (Naver movie)

# with open('movie_detail.json', 'r', encoding = 'utf-8') as f :
#     movie_detail = json.load(f)
# movie_rating_list = pd.read_csv('movie_rating_list.csv')

for i in range(len(movie_rating_list)) :
    url = 'https://openapi.naver.com/v1/search/movie.json'
    
    mv = movie_detail[movie_rating_list.iloc[i, 0]]
    payload = {
        'query' : mv['movieNm']
        , 'display' : 10
    }
    
    headers = {
        'X-Naver-Client-Id' : 'V8LONuGkI0U72r0RY3yo',
        'X-Naver-Client-Secret' : 'VKW2Diwq5d'
    }
    
    if (movie_rating_list.iloc[i, 1] == 0) | (movie_rating_list.iloc[i, 1] == 'NA') :
        # print(movie_rating_list.iloc[i, 0], mv['movieNm'] + mv['movieNmEn'])
        try :
            req = requests.get(url, headers = headers, params = payload)
            req.json()['total']
        except :
            movie_rating_list.iloc[i, 1] = 'NA'
        else :
            if(req.json()['total'] == 1) :
                movie_rating_list.iloc[i, 1] = req.json()['items'][0]['userRating']
            else :
                d = movie_detail[movie_rating_list.iloc[i, 0]]['directors']
                flag = False
                for jj in d :
                    for j in req.json()['items'] :
                        if(j['director'][:-1] == jj['peopleNm']) :
                            movie_rating_list.iloc[i,1]=j['userRating']
                            flag = True
                        if(flag == True) :
                            break
                    if(flag == True) :
                        break

[{'title': '<b>솔로몬 왕의 보고</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=13074', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0130/A3074-01.jpg', 'subtitle': "King Solomon's Mines", 'pubDate': '1950', 'director': '콤프톤 베넷|앤드류 마튼|', 'actor': '데보라 카|스튜어트 그레인저|리차드 칼슨|휴고 하스|로웰 길모어|키무르시|시리아크|세카리온고|바지가|', 'userRating': '6.83'}, {'title': '<b>솔로몬</b> 왕의 광산', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=15671', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0156/15671_P01_125747.jpg', 'subtitle': "King Solomon's Mines", 'pubDate': '1937', 'director': '로버트 스티븐슨|', 'actor': '폴 로브슨|세드릭 하드윅|', 'userRating': '0.00'}]


In [64]:
movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

# movie_data

## movie_detail.json
- Jupyter
- movieCd  
- movieNm  
- movieNmEn  
- movieNmOg ''  
- showTm '144'  
- prdtYear '2016'  
- openDt '20160601'  
- prdtStatNm '개봉'  
- typeNm  
- nations  
    * nationNm  
- genres  
    * genreNm  
- diretors  
    * peopleNm  
    * peopleNmEn  
- actors  
    * peopleNm  
    * peopleNmEn  
    * cast  
    * castEn  
- showTypes  
    * showTypeGroupNm  
    * showTypeNm  
- companys  
    * companyCd  
    * companyNm  
    * companyEn  
    * companyPartNm '제작사'  
- audits  
    * auditNo '2016-MF00708'  
    * watchGradeNm '청소년관람불가'  
- staffs  
    * peopleNm  
    * peopleNmEn  
    * staffRoleNm  
  
## movie_boxoffice.csv  
- rnum  
- rank  
- rankInten  
- rankOldAndNew   
- movieCd   
- movieNm  
- openDt  
- salesAmt  
- salesShare  
- salesInten  
- salesChange  
- salesAcc  
- audiCnt  
- audiInten  
- audiChange  
- audiAcc  
- scrnCnt  
- showCnt 
- CurrentDate
- multi  
- Nation  

### build movie_data

In [2]:
movie_boxoffice = pd.read_csv('movie_boxoffice.csv')

movie_group = movie_boxoffice.groupby(['movieCd'])
movie_data = pd.DataFrame({
                        'movieNm' : movie_group['movieNm'].max(),
                        'salesAcc' : movie_group['salesAcc'].max(),
                        'salesAvg' : movie_group['salesAmt'].mean(),
                        'screen' : movie_group['scrnCnt'].sum(),
                        'show' : movie_group['showCnt'].sum(),
                        'audienceAcc' : movie_group['audiAcc'].max(),
                        'audienceAvg' : movie_group['audiCnt'].mean(),
                        'multi' : movie_group['multi'].max(),
                        'nation' : movie_group['Nation'].max(),
                        'openDate' : movie_group['openDt'].min()
                        }).reset_index()

top10_sales = movie_data.sort_values(by = 'salesAcc', ascending = False).head(10).reset_index()
top10_audiences = movie_data.sort_values(by = 'audienceAcc', ascending = False).head(10).reset_index()

In [101]:
top10_sales

,index,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate
0,1458,20129370,명량,135756027310,1.190767e+09,40563,188579,17615437,154508.368421,N,K,2014-07-30
1,836,20090834,아바타,125304346000,2.672213e+07,619,1931,13338863,2380.333333,Y,F,2009-12-17
2,3269,20150976,신과함께-죄와 벌,115705891137,1.428419e+09,55203,213194,14411477,177911.975309,N,K,2017-12-20
3,2117,20137048,국제시장,110938092730,6.338978e+08,52226,212606,14262922,81495.977143,N,K,2014-12-17
4,2874,20148048,베테랑,105168923250,1.001502e+09,43995,199090,13414136,127735.523810,N,K,2015-08-05
5,6040,20186202,신과함께-인과 연,102664946909,1.425758e+09,44953,179889,12274846,170464.736111,N,K,2018-08-01
6,5394,20177478,어벤져스: 인피니티 워,99915606269,1.331828e+09,56248,240284,11211428,149440.826667,N,F,2018-04-25
7,3006,20148851,암살,98467187781,9.376947e+08,42924,175121,12706663,120997.695238,N,K,2015-07-22
8,4213,20162869,택시운전사,95868560649,9.129971e+08,46176,184080,12189141,116080.419048,N,K,2017-08-02
9,1263,20124034,도둑들,93667072500,6.140000e+05,2,5,12983821,154.000000,N,K,2012-07-25


In [103]:
top10_audiences

,index,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate
0,1458,20129370,명량,135756027310,1.190767e+09,40563,188579,17615437,154508.368421,N,K,2014-07-30
1,3269,20150976,신과함께-죄와 벌,115705891137,1.428419e+09,55203,213194,14411477,177911.975309,N,K,2017-12-20
2,2117,20137048,국제시장,110938092730,6.338978e+08,52226,212606,14262922,81495.977143,N,K,2014-12-17
3,2874,20148048,베테랑,105168923250,1.001502e+09,43995,199090,13414136,127735.523810,N,K,2015-08-05
4,836,20090834,아바타,125304346000,2.672213e+07,619,1931,13338863,2380.333333,Y,F,2009-12-17
5,1263,20124034,도둑들,93667072500,6.140000e+05,2,5,12983821,154.000000,N,K,2012-07-25
6,1402,20127593,7번방의 선물,91433282670,7.199427e+08,37820,166795,12811435,100876.598425,N,K,2013-01-23
7,3006,20148851,암살,98467187781,9.376947e+08,42924,175121,12706663,120997.695238,N,K,2015-07-22
8,1280,20124079,"광해, 왕이 된 남자",88908845769,5.840625e+05,16,36,12323510,256.937500,N,K,2012-09-13
9,6040,20186202,신과함께-인과 연,102664946909,1.425758e+09,44953,179889,12274846,170464.736111,N,K,2018-08-01


In [105]:
tmp_movie_data = movie_data

In [113]:
tmp_movie_data['salesPerAudit'] = movie_data['salesAcc']/movie_data['audienceAcc']

top10_salesPerAudit = tmp_movie_data.sort_values(by = 'salesPerAudit', ascending = False).head(10).reset_index()

NameError: name 'movie_data_table' is not defined

In [115]:
tmp_movie_data.head(5)

,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate,salesPerAudit
0,19278002,제너럴,857000,2.680000e+05,1,1,177,40.00,N,F,,4841.807910
1,19288001,잔다르크의 수난,2192000,7.306667e+05,3,18,1086,362.00,N,F,,2018.416206
2,19308006,모로코,3535000,5.387500e+05,4,19,1642,268.75,N,F,,2152.862363
3,19328008,스카페이스,4104000,1.026000e+06,4,16,2032,508.00,Y,F,,2019.685039
4,19338008,뮤지컬 42번가,5201000,7.556000e+05,5,25,2579,374.80,Y,F,,2016.673129


In [118]:
movie_detail['20144444']

{'movieCd': '20144444',
 'movieNm': '아가씨',
 'movieNmEn': 'The Handmaiden',
 'movieNmOg': '',
 'showTm': '144',
 'prdtYear': '2016',
 'openDt': '20160601',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '스릴러'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '박찬욱', 'peopleNmEn': 'PARK Chan-wook'}],
 'actors': [{'peopleNm': '김민희',
   'peopleNmEn': 'KIM Min-hee',
   'cast': '아가씨',
   'castEn': ''},
  {'peopleNm': '김태리', 'peopleNmEn': 'KIM Tae-ri', 'cast': '하녀', 'castEn': ''},
  {'peopleNm': '하정우', 'peopleNmEn': 'HA Jung-woo', 'cast': '백작', 'castEn': ''},
  {'peopleNm': '조진웅',
   'peopleNmEn': 'CHO Jin-woong',
   'cast': '이모부',
   'castEn': ''},
  {'peopleNm': '김해숙',
   'peopleNmEn': 'KIM Hae-sook',
   'cast': '여집사',
   'castEn': ''},
  {'peopleNm': '문소리', 'peopleNmEn': 'MOON So-ri', 'cast': '이모', 'castEn': ''},
  {'peopleNm': '최병모',
   'peopleNmEn': 'CHOE Byeong-mo',
   'cast': '손님',
   'castEn': ''},
  {'peopleNm': '김인우', 'peopleNmEn': 'K

In [264]:
movie_data[movie_data['movieCd']=='19278002']['multi'].iloc[0]

'N'

In [2]:
with open('movie_data_table_multi_y.json', 'r') as f :
    mdtm = json.load(f)


In [4]:
len(mdtm['actor'].items())

7483

In [7]:
## movie_data_key / only multi = 'N'
# actors
# directors
# genreNm
# nationNm
# showTypeNm
# typeNm
# watchGradeNm

with open('movie_detail.json', 'r', encoding = 'utf-8') as mdf :
    movie_detail = json.load(mdf)



def construct_movie_key(ref_md) :
    a_key = 0
    d_key = 0
    g_key = 0
    n_key = 0
    s_key = 0
    t_key = 0
    w_key = 0
    actor_list = []
    director_list = []
    genre_list = []
    nation_list = []
    showtype_list = []
    type_list = []
    grade_list = []
    
    result_list = []
    header_list = ['movieCd', 'actorNm', 'genreNm', 'nationNm', 'showTypeNm', 'typeNm', 'watchGradeNm']
    # print(ref_md['20144444'])
    for i in ref_md :
        # print(i)
        ## actor
        mm = movie_data[movie_data['movieCd'] == i]
        if(mm['multi'].iloc[0] == 'N') & (mm['nation'].iloc[0] == 'K') :
            try :
                for j in ref_md[i]['actors'] :
                    if j['peopleNm'] not in actor_list :
                        print(i + j['peopleNm'])
                        actor_list.append(j['peopleNm'])
                    # print(j)
            except :
                print("Error ! | actor")

            ## director
            try :
                for j in ref_md[i]['directors'] :
                    if j['peopleNm'] not in director_list :
                        director_list.append(j['peopleNm'])
                    # print(j)
            except :
                print("Error ! | director")

            ## genre
            try :
                for j in ref_md[i]['genres'] :
                    if j['genreNm'] not in genre_list :
                        genre_list.append(j['genreNm'])
                    # print(j)
            except :
                print("Error ! | genre")

            ## nation
            try :
                for j in ref_md[i]['nations'] :
                    if j['nationNm'] not in nation_list :
                        nation_list.append(j['nationNm'])
                    # print(j)
            except :
                print("Error ! | nation")

            ## showtype
            for j in ref_md[i]['showTypes'] :
                if j['showTypeGroupNm'] not in showtype_list :
                    showtype_list.append(j['showTypeGroupNm'])
                # print(j)

            ## type
            try :
                # print(ref_md[i]['typeNm'])
                if ref_md[i]['typeNm'] not in type_list :
                    type_list.append(ref_md[i]['typeNm'])
            except :
                print("Error ! | type")


            ## grade
            try :
                for j in ref_md[i]['audits'] :
                    if j['watchGradeNm'] not in grade_list :
                        grade_list.append(j['watchGradeNm'])
                    # print(j)
            except :
                print("Error ! | grade")
        
        
    # return pd.DataFrame(result_list, columns = header_list)
    return (actor_list, director_list, genre_list, nation_list, showtype_list, type_list, grade_list)

movie_data_key = construct_movie_key(movie_detail)

19460021유계선
19460021전창근
19460021송억
19540024이향
19540024윤인자
19550028이민자
19560001황해남
19560001지학자
19560023박노식
19560023황남
19560023허장강
19560023최강
19580061김지미
19580061김정훈
19580069김승호
19580069최남현
19580069최은희
19580069김진규
19588024장민호
19598036조미령
19598036이수련
19600050황정순
19600050황해
19600050윤일봉
19600051문정숙
19608020김혜정
19608020윤정란
19608043도금봉
19608043엄앵란
19628013신영균
19628028김희갑
19660041고은아
19678075신성일
19678075문희
19688114남궁원
19698128윤정희
19698128사미자
19698128김동원
19698128한은진
19710022남정임
19720009김성원
19720034나유
19720034남석훈
19720109전계현
19720109최정민
19720152나훈아
19728042김지수
19728042이영옥
19730023이준구
19730023모영
19730092링원
19730092진봉진
19730122최무룡
19740037최성호
19740037윤미라
19740137이순재
19740137박병호
19740137이효춘
19748218문숙
19750002양정화
19750018신영일
19750018김희라
19750018김창숙
19750018정윤희
19750021박근형
19750082이정길
19750091이대근
19750091여수진
19750141서미경
19750141독고성
19760004진유영
19760004유영국
19760004김지혜
19760017이덕화
19760017임예진
19760017김윤경
19760017김보연
19768232왕호
19768232황태수
19768232조춘
19770003장동휘
19770003안보영
19770003최봉
19770003장혁
197700

20080897김정철
20080897박암
20080897최윤석
20080897이화시
20080897박정자
20080899김 만
20080902김해숙
20081003김옥빈
20081003박인환
20081003손종학
20081003황우슬혜
20081066한소룡
20081067김정옥
20081146박지훈
20081146오지영
20081391이종혁
20081391서우
20081391서영주
20081391김희영
20081391박지수
20081391박찬욱
20081391정준수
20081391장진영
20081391장혜진
20081391양현진
20081391배성우
20081555김석균
20081555황병국
20081555박정범
20081555박영덕
20081740김동령
20088318천호진
20088318김효진
20088318지서윤
20088318예학영
20088506한혜진
20088506임슬옹
20088506배수빈
20088506이경영
20088506장광
20088506천우희
20088506이현걸
20088506전종석
20088506지성용
20088506김병오
20088506이상훈
20088506정현진
20088506이미도
20088506김도윤
20088506김태준
20090060최계영
20090074엄정화
20090074이민기
20090074강대규
20090074김원영
20090074김유빈
20090074신정원
20090074성유경
20090074박재홍
20090074황인준
20090074나승현
20090074김인호
20090074근휘
20090074김재경
20090074장원준
20090074임동우
20090074김정곤
20090074정종원
20090074이지애
20090074이태영
20090074장명갑
20090074천보근
20090074박영수
20090237구교환
20090237정려원
20090237강문정
20090237박보영
20090237이성배
20090237임수진
20090237최소영
20090237이규형
20090237양미경
20090237이정애
2009024

20126090정호빈
20126090장희수
20126091이종석
20126091신기원
20126091윤종구
20126091윤희원
20126091이도엽
20126091권영각
20126091김길동
20126091정준식
20126091조훈휘
20126091표유신
20126091하정수
20126091한만영
20126091노준표
20126091문혜진
20126091전신환
20126091김강현
20126543하은채
20126583조안
20126583이광수
20126583이청미
20126583이효선
20126583조선아
20126583최지연
20126583강두
20126671이태리
20126671남경읍
20126671정동석
20126671조원철
20126671김도연
20126671김선간
20126672박기웅
20126672손현주
20126672박혜숙
20126672홍경인
20126672이채영
20126672박은빈
20126672최우식
20126672이재남
20126672장하은
20126672장하진
20126672황의준
20126672송경호
20126672이명규
20126674크리스 에반스
20126674에드 해리스
20126674존 허트
20126674틸다 스윈튼
20126674제이미 벨
20126674옥타비아 스펜서
20126674이완 브렘너
20126674알리슨 필
20126674허찬
20127185엄태구
20127185류혜영
20127191박주희
20127191안재홍
20127201조운
20127593박신혜
20127593갈소원
20127593장호준
20127593김동민
20127593박수진
20127593박유경
20127593박정배
20127595배성민
20127595백주연
20127595서정훈
20127595송은영
20127595안성민
20127595이소민
20127595이정수
20127595이준수
20127595권혁석
20127595임병우
20127595정대곤
20127595전도영
20127595김정섭
20127595차준호
20127595최정선
20127595허

20139401이지은
20139401김도형
20139401김민호
20139401김범준
20139401김진성
20139401문호진
20139401민정기
20139401최미라
20139401고원희
20139402홍수아
20139402서지석
20139402한수아
20139402주아성
20139402성지영
20139402송윤아
20139402오승진
20139402이종우
20139402이진솔
20139422박현복
20139422이종호
20139441조진우
20139492백수장
20139550최혁
20139626강혜정
20139861고동업
20139861황경
20139862유상재
20139881김영광
20139881신현탁
20139881장세준
20139881손지훈
20139881정승호
20139881윤보현
20139881조유진
20139881조현주
20139881최영
20139881최현영
20139881홍수정
20139881이예진
20139881이정아
20139881김기철
20139881김수정
20139881김영
20139881김종현
20139881김하경
20139881박선주
20139881여윤호
20139881정재훈
20139881윤원준
20139964장래군
20139964허진
20139981주원
20139981강남현
20139981임우철
20139981장원
20139981신비감
20139981조영헌
20139981차승훈
20139981최규호
20139981이성동
20139981이준규
20139981이지영
20139981김미나
20139981김미라
20139981김미숙
20139981김소라
20139981김영주
20139981김초롱
20140081유라성
20140081강기영
20140081박재훈
20140081허종우
20140081이완섭
20140112추선
20140121루이 들로르
20140121록산느 르 텍시에
20140121로드 자누와
20140121카미유 루
20140144조정숙
20140144황태양
20140144김은지
20140144박선
20140144박솔지

20150964오지홍
20150964조진규
20150964최경미
20150964최경식
20150964최창국
20150964이석
20150964한성관
20150964이유진
20150964박지후
20150966서현진
20150966선우용녀
20150966김사희
20150966장도윤
20150966김강훈
20150966이창희
20150966정보연
20150966이재우
20150968오연서
20150968임진영
20150968박진하
20150968전송이
20150968방추성
20150968백상훈
20150968서은정
20150968조규영
20150968안승호
20150968조수지
20150968조희윤
20150968이경선
20150968이선미
20150968이영화
20150968함지후
20150969강민혁
20150969최민호
20150969박선영
20150969이한서
20150969윤송아
20150969이선빈
20150969이소현
20150969김정겸
20150969배지혜
20150969이지민
20150969신혜경
20150969유다영
20150969윤예원
20150969민성주
20150969장지선
20150969정민규
20150969정인영
20150969강현우
20150969조성환
20150969최준호
20150969김서현
20150969김성훈
20150971서수민
20150971이강재
20150972곽시양
20150972전석호
20150972이형미
20150972정준화
20150972윤환
20150972최문석
20150972홍민희
20150976최용훈
20150976강소백
20150976정수윤
20150976이지향
20150977하석진
20150977지윤호
20150977이한승
20150977이현서
20150977장원진
20150977전호성
20150977송창현
20150979정상훈
20150979김소현
20150979박주미
20150979김재욱
20150979박윤식
20150979박혜영
20150979송민수
20150979신동주
20150979오세영
20150

20162662제동화
20162662이강연
20162662김종인
20162701임형순
20162721엄국현
20162722배정남
20162722염진환
20162722최형욱
20162722손여은
20162852윤기원
20162852조용범
20162852최효주
20162869토마스 크레취만
20162869이창섭
20162869이호균
20162869오석규
20162869황민형
20162869김신웅
20162871이기홍
20162871이준범
20162871정병진
20162871정연서
20162871이상구
20162871이상준
20162871홍윤기
20162871정재윤
20162924유소현
20162924김유연
20162924서혜림
20162924정찬범
20163152수지
20163174정겨운
20163174이태인
20163174허충호
20163199이혜빈
20163218김정균
20163218장두이
20163319박재완
20163350이지우
20163350이유희
20163443정숙희
20163521신민수
20163521윤수현
20163521이정주
20163521이지수
20163521최승
20163665도모새
20163721강현중
20164001정원
20164174임보라
20164174박재웅
20164174안세진
20164174조청원
20164174지성훈
20164174최상재
20164174이윤하
20164281이세창
20164304조한철
20164304차미영
20164304윤진욱
20164304최예슬
20164304서이안
20164321하경
20164323아이린
20164323정승길
20164323이명하
20164403최운정
20164404김세인
20164404이재욱
20164424정규수
20164424이서원
20164424곽동연
20164424최서윤
20164424이신우
20164424이원태
20164424홍재식
20164424황정용
20164541정선민
20164541도모세
20164541정석
20164741고찬우
20164844허준호
20164844박지호
2016

In [270]:
len(movie_data_key)

7

In [8]:
md_key_list = ['actor', 'director', 'genre', 'nation', 'showType', 'type', 'grade']
movie_data_table = dict()
for j in range(0,7) :
    cnt = 0
    md_dict = dict()
    for i in movie_data_key[j] :
        md_dict[i] = json.dumps(int(cnt))
        cnt = cnt + 1
    movie_data_table[md_key_list[j]] = md_dict

In [10]:
len(movie_data_table['actor'])

5198

In [11]:
with open('movie_data_table_nonmulti_kr_2.json', 'w') as output :
    json.dump(movie_data_table, output)

### movie_data_table.csv

In [276]:
## movie_data_key
# actors
# directors
# genreNm
# nationNm
# showTypeNm
# typeNm
# watchGradeNm

with open('movie_detail.json', 'r', encoding = 'utf-8') as mdf :
    movie_detail = json.load(mdf)



def construct_movie_key(ref_md) :
    a_key = 0
    d_key = 0
    g_key = 0
    n_key = 0
    s_key = 0
    t_key = 0
    w_key = 0
    actor_list = []
    director_list = []
    genre_list = []
    nation_list = []
    showtype_list = []
    type_list = []
    grade_list = []
    
    result_list = []
    header_list = ['movieCd', 'actorNm', 'genreNm', 'nationNm', 'showTypeNm', 'typeNm', 'watchGradeNm']
    # print(ref_md['20144444'])
    for i in ref_md :
        # print(i)
        ## actor
        try :
            for j in ref_md[i]['actors'] :
                if j['peopleNm'] not in actor_list :
                    print(i + j['peopleNm'])
                    actor_list.append(j['peopleNm'])
                # print(j)
        except :
            print("Error ! | actor")
        
        ## director
        try :
            for j in ref_md[i]['directors'] :
                if j['peopleNm'] not in director_list :
                    director_list.append(j['peopleNm'])
                # print(j)
        except :
            print("Error ! | director")
            
        ## genre
        try :
            for j in ref_md[i]['genres'] :
                if j['genreNm'] not in genre_list :
                    genre_list.append(j['genreNm'])
                # print(j)
        except :
            print("Error ! | genre")
        
        ## nation
        try :
            for j in ref_md[i]['nations'] :
                if j['nationNm'] not in nation_list :
                    nation_list.append(j['nationNm'])
                # print(j)
        except :
            print("Error ! | nation")
        
        ## showtype
        for j in ref_md[i]['showTypes'] :
            if j['showTypeGroupNm'] not in showtype_list :
                showtype_list.append(j['showTypeGroupNm'])
            # print(j)
        
        ## type
        try :
            # print(ref_md[i]['typeNm'])
            if ref_md[i]['typeNm'] not in type_list :
                type_list.append(ref_md[i]['typeNm'])
        except :
            print("Error ! | type")
            
        
        ## grade
        try :
            for j in ref_md[i]['audits'] :
                if j['watchGradeNm'] not in grade_list :
                    grade_list.append(j['watchGradeNm'])
                # print(j)
        except :
            print("Error ! | grade")
        
        
    # return pd.DataFrame(result_list, columns = header_list)
    return (actor_list, director_list, genre_list, nation_list, showtype_list, type_list, grade_list)

movie_data_key = construct_movie_key(movie_detail)

19278002버스터 키튼
19278002마리온 맥
19308006게리 쿠퍼
19308006마를렌 디트리히
19328008폴 무니
19358005로버트 도냇
19358005매들린 캐롤
19358007찰스 로튼
19358007클라크 게이블
19358007프랑코트 톤
19368015로버트 테일러
19368015그레타 가르보
19378007폴 로버슨
19378007세드릭 하드윅
19378007로랜드 영
19378023자넷 게이노
19378023프레드릭 마치
19380009캐리 그랜트
19380009캐서린 헵번
19388001에롤 플린
19388001올리비아 드 하빌랜드
19388005진 아서
19388005라이오넬 배리모어
19398001주디 갈란드
19398005헨리 폰다
19398005앨리스 브래디
19398015로렌스 올리비에
19398015메를 오베론
19408005제임스 스튜어트
19408011그리어 가슨
19418007오손 웰즈
19418007도로시 코밍고어
19418007조셉 코튼
19418009바바라 스탠윅
19420001험프리 보가트
19420001잉그리드 버그만
19420001폴 헌레이드
19428001제임스 카그니
19428001조안 레슬리
19428006랜돌프 스콧
19428006존 웨인
19428012캐롤 롬바드
19428012잭 베니
19438005제니퍼 존스
19438005찰스 빅포드
19438006브루스 버네트
19438006J. 캐롤 네이쉬
19438008수잔나 포스터
19438008넬슨 에디
19438010마들레느 솔로뉴
19438010장 마레
19448003빙 크로스비
19448003배리 피츠제럴드
19448005찰스 보이어
19448011레슬리 뱅크스
19448011에스몬드 나이트
19448011로버트 뉴튼
19458003로저 리브시
19458003데보라 카
19458005셀리아 존슨
19458005트레버 하워드
19458008레이 밀랜드
19458008제인 와이먼
19460021유계선
19460021전창근
19460021송억
1

20068576문채원
20068576조재윤
20068576김슬기
20068576이해영
20068576이연두
20068576박민우
20068576이창해
20068576인교진
20068576임영
20068576박태영
20068576정경옥
20068576정라엘
20068576오형근
20068576이범희
20068576이은정
20068576이정은
20068659야마구치 캇페이
20068730클라이브 오웬
20068788파즈 베가
20068788조나 힐
20070008휴 잭맨
20070054조경래
20070054김화수
20070054조명수
20070054홍성택
20070054홍성화
20070054조성래
20070054김희숙
20070054김희윤
20070054박용일
20070054조영래
20070054박지연
20070054박지영
20070054조창휘
20070054변재훈
20070054강성룡
20070054서정화
20070054김대황
20070054김일수
20070054윤병철
20070054장광수
20070054장영진
20070054정미나
20070054박선화
20070054박세영
20070054박순애
20070054박애리
20070054정시영
20070054오희연
20070054정지수
20070054김정복
20070054김순
20070054오경준
20070054김지연
20070068울리히 뮤흐
20070068마르티나 게덱
20070068세바스티안 코치
20070068울리히 터커
20070083벤 위쇼
20070099조경화
20070099김효신
20070099신경호
20070099조영재
20070099서명희
20070099우영제
20070099문현
20070099박근영
20070099오승은
20070099송순섭
20070099온정준
20070099김양휘
20070099이청준
20070114이세영
20070114김윤아
20070114김명환
20070114조용상
20070114김민숙
20070114임지환
20070114박주용
20070115김종수
20070115오만석
20

20112561짐 스터게스
20112589소지섭
20112589한효주
20112589최재훈
20112589김희승
20112589신용철
20112589신은진
20112589최제헌
20112589김현철
20112589최철수
20112589최혜진
20112589심철민
20112589표형렬
20112589안재민
20112589백지영
20112589권지훈
20112589강성대
20112589박갑진
20112589박동규
20112589김성종
20112589인지연
20112589장성희
20112589오정택
20112589최은혜
20112589이길우
20112589윤종화
20112695김효숙
20112695조병찬
20112695조세현
20112695신창수
20112695정한진
20112695강별
20112695박효주
20112695서대현
20112695서왕석
20112695김광섭
20112695차선영
20112695천영암
20112695최명경
20112695윤경호
20112695전승엽
20112695임정환
20112695김신우
20112695이경숙
20112695이유나
20112695성유빈
20112703톰 하디
20112768타이론 파워
20112774톰 이웰
20112775리아넬라 카렐
20112776전영선
20112781정희성
20112781이재희
20112781노형석
20112781박지만
20112781신아가
20112781신준호
20112791이천희
20112791김새론
20112791김아론
20112791정기철
20112791정민우
20112791정성룡
20112791정우상
20112791정재욱
20112791정지민
20112791최가현
20112791최성우
20112791김민수
20112791최영진
20112791이윤경
20112791박경균
20112795정지영
20112795임권택
20112795이진훈
20112795이두용
20112808정기용
20112808승효상
20112808유걸
20112841김영필
20112841신동미
20112841방영배
201128

20123741김기리
20123781봉소악
20123781장효전
20123821정범균
20123821김지민
20124013최고요
20124013이재식
20124013나경찬
20124013박진하
20124013강민아
20124024할리 베리
20124024휴고 위빙
20124031윤다경
20124031심은진
20124031서효명
20124031김동범
20124031정완희
20124031정재필
20124031정희원
20124031김동민
20124031최종렬
20124031박정혁
20124031박형진
20124031송동규
20124034이심결
20124034증국상
20124034장동
20124034장현문
20124034정윤헌
20124034한규석
20124034홍원기
20124039도지한
20124039차인표
20124039조민아
20124039권현상
20124039박정학
20124039임경만
20124039장혜리
20124039조윤경
20124039배진호
20124039안용우
20124039이창용
20124043신이
20124043민아
20124043정애연
20124043경수진
20124043이창대
20124043박준서
20124043김나온
20124044박지빈
20124044김보라
20124044임성호
20124044조정호
20124044이민주
20124044김진태
20124044이주신
20124044박현우
20124044송지선
20124044신동진
20124044우보경
20124044윤상윤
20124047윤시윤
20124047강경덕
20124047박효준
20124047정유신
20124047정진욱
20124047조세영
20124047이민영
20124047김진아
20124050차수연
20124050임남숙
20124050임순옥
20124050장대윤
20124050황남희
20124050박지혜
20124050송성원
20124050오기민
20124050김성태
20124054장서이
20124054한민혁
20124054이승주
20124054최광락
20124054오영석
201

20130703알렉상드르 나혼
20130704찰리 크리드 마일스
20130704레오 그레고리
20130704윌 폴터
20130704리즈 화이트
20130705프랑소아 다미앙
20130705앤느 폴리세비치
20130705세르지 로페즈
20130705잔 해멘넥커
20130706이수정
20130706정은우
20130706신유주
20130706이주엽
20130706주종휘
20130708케이티 창
20130708이스라엘 브로우사드
20130710테렌스 하워드
20130784에밀리 코제트
20130784오렐리 뒤퐁
20130801이타이 슈체르베크
20130801토마슈 솝자크
20130821제임스 프레체빌
20130821벤 멘델슨
20130841슈퍼주니어
20130929강경래
20130929강나연
20130929정광수
20130929최종태
20130929하재민
20130929이아영
20130929김남호
20130929황성우
20130929백승철
20130929조민준
20130952렉스 해리슨
20130952콘스탄스 커밍스
20130961킹스턴 루디스카
20130961성낙원
20130961손형식
20130961조수범
20130961한승진
20130982아우구스티노
20130982타실라
20130982호앙 타인
20130982필립 곤잘레스
20130982와리우라 브후아이야
20130982강가 바스넷
20130982브라욘 쁘쿤톳
20130982캐서린 앤 코르테자
20130982초웁 칸피아루
20130982셰크할 마문
20130982쉬첸밍
20130982셰르조드 아크바로브
20130982오운 라모우
20130982응우옌 반 응우옌
20131001마리아노 마르티네즈
20131001루이사나 로필라토
20131001김민국
20131001송지아
20131003돌프 룬드그렌
20131003스티브 오스틴
20131081줄리아 사라 스톤
20131081마차 그레논
20131081길 벨로우스
20131082조니 루이스
20131082알렉산드라 홀든
20131083케이티 롯츠
20131083캐스

...

20157584탁원정
20157584권문정
20157587김성근
20157587설재훈
20157587이승엽
20157587이승재
20157588콜린 모건
20157591딘 케인
20157591리차드 리엘
20157602브누와 뽀엘부르드
20157603제임스 프리드슨-잭슨
20157603헤이즈 웰포드
20157631장혜민
20157631조혁찬
20157631최승주
20157631노일환
20157781피비 폭스
20157781헬렌 맥크로리
20157782쉐인 하퍼
20157782트리샤 라파체
20157782코리 올리버
20157782하딜 싯투
20157881레베카 퍼거슨
20157883시손 쥰
20157883세키네 츠토무
20157986한고은
20157986배그린
20157986신정선
20157986전현숙
20157986한성아
20157989프랑수아즈 도를리크
20157998롭 브라이든
20158019민복기
20158020패트리샤 닐
20158022박성대
20158042모아 감멜
20158042아니타 월
20158042시몬 J. 베르거
20158042에리크 룬드퀴비스트
20158044미하일 고미아쉬빌리
20158044다치 오르벨라쉬빌리
20158061할룩 빌기너
20158061멜리사 소젠
20158061드멧 아크백
20158061네잣 이슬러
20158068손가람
20158068전예은
20158068조명희
20158068조미연
20158069배진홍
20158082로랑 라피트
20158091박리나
20158091유지원
20158150토비 켑벨
20158161정혜림
20158161안수빈
20158161하강웅
20158161김수빈
20158182김정연
20158182정연
20158182이연서
20158182이원
20158184피터 마크 리치만
20158192마일로 파커
20158192앙케 엥겔케
20158203김영삼
20158203아리
20158203안광연
20158203명지훈
20158240주영
20158241피터 딘클리지
20158241애슐리 벤슨
20158242홍은

20162607케이트 델 카스틸로
20162608김용옥
20162621크리스 J. 머레이
20162621브릿 쇼
20162621아이비 조지
20162629히노 사토시
20162662제동화
20162662이강연
20162662김종인
20162681하나모리 유미리
20162681사쿠라 아야네
20162701임형순
20162721엄국현
20162722배정남
20162722염진환
20162727김가령
20162727미즈키 나나
20162727후루타 아라타
20162741트로이 도나휴
20162741도로시 맥과이어
20162762길예르모 프란셀라
20162762피터 란자니
20162765아이스 큐브
20162765레지나 홀
20162765안소니 앤더슨
20162765이브
20162765숀 패트릭 토마스
20162765가르셀라 보베
20162765우카쉬 암부카
20162765트로이 가리티
20162765디레이 데이비스
20162829제이슨 도링
20162829캐서린 라나사
20162829질리언 머레이
20162829데이빗 오도넬
20162829팀 웨어
20162846조셀린 도나휴
20162846디에고 보네타
20162852윤기원
20162852조용범
20162852최효주
20162869이창섭
20162869이호균
20162869오석규
20162869황민형
20162869김신웅
20162871이준범
20162871정병진
20162871정연서
20162871이상구
20162871홍윤기
20162873로자리오 도슨
20162873마이클 세라
20162875류호연
20162902마크 게티스
20162924유소현
20162924김유연
20162924정찬범
20162925로렌 코핸
20162926마리안느 세이지브레트
20162926CCH 파운더
20162926잭 팰런스
20162928동자건
20162928안탁령
20162928임백굉
20162928오양니니
20162941프란카 포텐테
20162941스털링 제린스
20162962리오넬 메시
20162962호셉 과르디올라
2016296

20170443그웬돌린 크리스티
20170443켈리 마리 트란
20170509이토 카나에
20170513신세휘
20170513양기원
20170513강민준
20170525노넨 레나
20170525호소야 요시마사
20170561채드윅 보스만
20170561다네이 거라이라
20170561레티티아 라이트
20170561다니엘 칼루야
20170561윈스턴 듀크
20170590윤재국
20170590최광일
20170607루이스 맥더겔
20170624사쿠라 유라
20170628아드리안 티티에니
20170656데이빗 하버
20170656가브리엘 유니온
20170656티.아이.
20170656살라 베이커
20170656스티브 콜터
20170656릭 레잇츠
20170669김지환
20170669이진
20170669변지은
20170669송지연
20170669이준성
20170669유은성
20170669차봉주
20170670이혜민
20170670최정인
20170741도미닉 웨스트
20170741바네사 커비
20170741마크 아놀드
20170761로라 프레폰
20170761저스틴 세럭스
20170761리사 쿠드로
20170787한가희
20170841진기주
20170841이광용
20170841김재훈
20170841김주은
20170841오수엽
20170841임정희
20170841김다정
20170841김동혁
20170841최혜림
20170902크래그 힐
20170921샤하브 호세이니
20170921타라네 앨리두스티
20170922이쿠타 토마
20170922키리타니 켄타
20170922미무라
20170922코이케 에이코
20170922에구치 노리코
20170922카키하라 린카
20170931리 반 클리프
20170932프랑코 네로
20170932우디 스토드
20170933에스더 윌리암스
20170941윤성모
20170941조민호
20170941박종훈
20170942문예원
20170942이시형
20170942서영민
20170942양도현
20170942김현창
20170942정원종
20170942김

20176169나카지마 켄토
20176169요시네 쿄코
20176169이시이 안나
20176169칸이치로
20176251권혁범
20176251이호수
20176251김상겸
20176345허윤철
20176421이장미
20176421우진영
20176511막스 리멜트
20176514카메나시 카즈야
20176514나카지마 토모코
20176561줄리아 옌체
20176562마이크 보겔
20176562에리카 크리스틴슨
20176562로버트 포스터
20176581박정섭
20176603짐 벨루시
20176666양민우
20176666진달래
20176717로라 베난티
20176741구민선
20176742크리스토퍼 애봇
20176761후쿠시 소타
20176761쿠도 아스카
20176761요시다 코타로
20176765김남오
20176765박준헌
20176781앤 헤이시
20176784호시노 겐
20176785시모다 쇼타
20176785사이토 소마
20176848재지 비츠
20176917말리 쉘튼
20176961카이라 세드윅
20176962안민상
20176971김기영
20176983이원찬
20176983심승한
20176983사성웅
20177005이예원
20177005이배
20177005배수현
20177005박성범
20177005한경미
20177005김민혜
20177041조배성
20177086멜라니 티에리
20177101김종신
20177104존 시나
20177104미구엘 앙겔 실베스트르
20177144피에르 브라소
20177144세르주 레지아니
20177181클레어 홀트
20177203노성균
20177223김태원
20177223박창수
20177223이영현
20177245여주영
20177245김민
20177245김윤지
20177245최소담
20177245홍가
20177266김다미
20177266윤희정
20177308유관형
20177308장민정
20177308정휘욱
20177308김보민
20177341강병걸
20177341김기용
20177341차진영
20177342양홍석
20177347알렉스

20184529나연
20184529모모
20184529사나
20184529지효
20184529미나
20184529다현
20184529채영
20184529쯔위
20184642장혜정
20184642유인서
20184678안준태
20184678남기주
20184678문득산
20184689김설
20184689이스마일 하지오글루
20184699아녜스 바르다
20184699제이알
20184762사카모토 류이치
20184801알렉스 울프
20184801밀리 샤피로
20184802캐서린 한
20184803이케다 슈이치
20184803우라야마 진
20184803긴가 반죠
20184821후루야 토오루
20184845패트릭 슈왈제네거
20184845퀸 쉐퍼드
20184845니콜라스 쿰베
20184962정재은
20184962양수빈
20184962주민준
20184962김소율
20184962정승원
20184962김준석
20184981고애리
20184981정선철
20184981조혜련
20185001김사라
20185001홍후백
20185046데브라 메싱
20185222빌리 하울
20185222앤-마리 듀프
20185341백규현
20185341김종면
20185341이하영
20185341모그
20185341장민재
20185341진영준
20185341권용석
20185341김영원
20185350박금분
20185350곽두조
20185350강금연
20185350안윤선
20185350박월선
20185350김두선
20185350이원순
20185350박복형
20185350주석희
20185484에카테리나 구세바
20185485라미 말렉
20185485조셉 마젤로
20185485마이크 마이어스
20186121죠 카이리
20186121사사키 미유
20186184크리스티앙 드 라 푸엔테
20186184제시카 세디엘
20186202김지안
20186202장재욱
20186281파멜라 애들론
20186281피터 쿨렌
20186282이와미 마나카
20186284박진용
20186321패트릭 후짓
20186321에단 엠브리
2

In [278]:
actor_table = movie_data_key[0]

md_key_list = ['actor', 'director', 'genre', 'nation', 'showType', 'type', 'grade']
movie_data_table = dict()
for j in range(0,7) :
    cnt = 0
    md_dict = dict()
    for i in movie_data_key[j] :
        md_dict[i] = json.dumps(int(cnt))
        cnt = cnt + 1
    movie_data_table[md_key_list[j]] = md_dict

In [24]:
len(movie_data_table['director'])

1386

In [23]:
len(movie_data_table['actor'])

5198

In [165]:
with open('movie_data_table.json', 'w') as output :
    json.dump(movie_data_table, output)

In [166]:
mdt = dict()

with open('movie_data_table.json', 'r') as f :
    mdt = json.load(f)

movie_data_table['actor']

{'버스터 키튼': '0',
 '마리온 맥': '1',
 '게리 쿠퍼': '2',
 '마를렌 디트리히': '3',
 '폴 무니': '4',
 '로버트 도냇': '5',
 '매들린 캐롤': '6',
 '찰스 로튼': '7',
 '클라크 게이블': '8',
 '프랑코트 톤': '9',
 '로버트 테일러': '10',
 '그레타 가르보': '11',
 '폴 로버슨': '12',
 '세드릭 하드윅': '13',
 '로랜드 영': '14',
 '자넷 게이노': '15',
 '프레드릭 마치': '16',
 '캐리 그랜트': '17',
 '캐서린 헵번': '18',
 '에롤 플린': '19',
 '올리비아 드 하빌랜드': '20',
 '진 아서': '21',
 '라이오넬 배리모어': '22',
 '주디 갈란드': '23',
 '헨리 폰다': '24',
 '앨리스 브래디': '25',
 '로렌스 올리비에': '26',
 '메를 오베론': '27',
 '제임스 스튜어트': '28',
 '그리어 가슨': '29',
 '오손 웰즈': '30',
 '도로시 코밍고어': '31',
 '조셉 코튼': '32',
 '바바라 스탠윅': '33',
 '험프리 보가트': '34',
 '잉그리드 버그만': '35',
 '폴 헌레이드': '36',
 '제임스 카그니': '37',
 '조안 레슬리': '38',
 '랜돌프 스콧': '39',
 '존 웨인': '40',
 '캐롤 롬바드': '41',
 '잭 베니': '42',
 '제니퍼 존스': '43',
 '찰스 빅포드': '44',
 '브루스 버네트': '45',
 'J. 캐롤 네이쉬': '46',
 '수잔나 포스터': '47',
 '넬슨 에디': '48',
 '마들레느 솔로뉴': '49',
 '장 마레': '50',
 '빙 크로스비': '51',
 '배리 피츠제럴드': '52',
 '찰스 보이어': '53',
 '레슬리 뱅크스': '54',
 '에스몬드 나이트': '55',
 '로버트 뉴튼': '56',
 '로저 리브시': '57',
 '데보라

In [43]:
movie_detail['20144444']

{'movieCd': '20144444',
 'movieNm': '아가씨',
 'movieNmEn': 'The Handmaiden',
 'movieNmOg': '',
 'showTm': '144',
 'prdtYear': '2016',
 'openDt': '20160601',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '스릴러'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '박찬욱', 'peopleNmEn': 'PARK Chan-wook'}],
 'actors': [{'peopleNm': '김민희',
   'peopleNmEn': 'KIM Min-hee',
   'cast': '아가씨',
   'castEn': ''},
  {'peopleNm': '김태리', 'peopleNmEn': 'KIM Tae-ri', 'cast': '하녀', 'castEn': ''},
  {'peopleNm': '하정우', 'peopleNmEn': 'HA Jung-woo', 'cast': '백작', 'castEn': ''},
  {'peopleNm': '조진웅',
   'peopleNmEn': 'CHO Jin-woong',
   'cast': '이모부',
   'castEn': ''},
  {'peopleNm': '김해숙',
   'peopleNmEn': 'KIM Hae-sook',
   'cast': '여집사',
   'castEn': ''},
  {'peopleNm': '문소리', 'peopleNmEn': 'MOON So-ri', 'cast': '이모', 'castEn': ''},
  {'peopleNm': '최병모',
   'peopleNmEn': 'CHOE Byeong-mo',
   'cast': '손님',
   'castEn': ''},
  {'peopleNm': '김인우', 'peopleNmEn': 'K

### movie_data_table_nonmulti_kr.csv

In [25]:
with open('movie_data_table_nonmulti_kr.json', 'r') as f :
    mdtnk = json.load(f)
    
with open('movie_data_table_multi_kr.json', 'r') as f :
    mdtmk = json.load(f)

In [36]:
len(mdtnk['actor'].items())

dict_items([('유계선', 0), ('전창근', '31400'), ('송억', 0), ('이향', 0), ('윤인자', '14700'), ('이민자', '1220000'), ('황해남', 0), ('지학자', 0), ('박노식', '28500'), ('황남', 0), ('허장강', '25700'), ('최강', 0), ('김지미', '82000'), ('김정훈', '652000'), ('김승호', '90900'), ('최남현', '6530'), ('최은희', '105000'), ('김진규', '67500'), ('장민호', '49500'), ('조미령', '51400'), ('이수련', 0), ('황정순', '22700'), ('황해', '212000'), ('윤일봉', '21600'), ('문정숙', '26200'), ('김혜정', '149000'), ('윤정란', '2290'), ('도금봉', '10200'), ('엄앵란', '162000'), ('신영균', '96100'), ('김희갑', '25000'), ('고은아', '576000'), ('신성일', '410000'), ('문희', '131000'), ('남궁원', '30900'), ('윤정희', '147000'), ('사미자', 0), ('김동원', '139000'), ('한은진', '10000'), ('남정임', 0), ('김성원', '164000'), ('나유', 0), ('남석훈', 0), ('전계현', '5140'), ('최정민', 0), ('나훈아', 0), ('김지수', '1510000'), ('이영옥', '25200'), ('이준구', 0), ('모영', 0), ('링원', 0), ('진봉진', 0), ('최무룡', '28600'), ('최성호', '48700'), ('윤미라', 0), ('이순재', '1080000'), ('박병호', 0), ('이효춘', 0), ('문숙', '92500'), ('양정화', '1330000'), ('신영일', 0), ('김희라', '32600')

In [30]:
for k in mdtnk['actor'] :
    mdtnk['actor'][k] = 0

for k in mdtmk['actor'] :
    try :
        if(mdtnk['actor'][k] == 0) :
            mdtnk['actor'][k]=mdtmk['actor'][k]
    except :
        continue

{'유계선': 0,
 '전창근': '31400',
 '송억': 0,
 '이향': 0,
 '윤인자': '14700',
 '이민자': '1220000',
 '황해남': 0,
 '지학자': 0,
 '박노식': '28500',
 '황남': 0,
 '허장강': '25700',
 '최강': 0,
 '김지미': '82000',
 '김정훈': '652000',
 '김승호': '90900',
 '최남현': '6530',
 '최은희': '105000',
 '김진규': '67500',
 '장민호': '49500',
 '조미령': '51400',
 '이수련': 0,
 '황정순': '22700',
 '황해': '212000',
 '윤일봉': '21600',
 '문정숙': '26200',
 '김혜정': '149000',
 '윤정란': '2290',
 '도금봉': '10200',
 '엄앵란': '162000',
 '신영균': '96100',
 '김희갑': '25000',
 '고은아': '576000',
 '신성일': '410000',
 '문희': '131000',
 '남궁원': '30900',
 '윤정희': '147000',
 '사미자': 0,
 '김동원': '139000',
 '한은진': '10000',
 '남정임': 0,
 '김성원': '164000',
 '나유': 0,
 '남석훈': 0,
 '전계현': '5140',
 '최정민': 0,
 '나훈아': 0,
 '김지수': '1510000',
 '이영옥': '25200',
 '이준구': 0,
 '모영': 0,
 '링원': 0,
 '진봉진': 0,
 '최무룡': '28600',
 '최성호': '48700',
 '윤미라': 0,
 '이순재': '1080000',
 '박병호': 0,
 '이효춘': 0,
 '문숙': '92500',
 '양정화': '1330000',
 '신영일': 0,
 '김희라': '32600',
 '김창숙': 0,
 '정윤희': 0,
 '박근형': '174000',
 '이정길': 0,
 '이대근': '34400',
 '여수

In [38]:
mdtmk_bak = mdtmk

In [39]:
with open('movie_data_table_multi_kr.json', 'w', encoding = 'utf-8') as output :
    json.dump(mdtmk, output)

In [31]:
with open('movie_data_table_nonmulti_kr.json', 'w', encoding = 'utf-8') as output :
    json.dump(mdtnk, output)

### movie.csv

In [243]:
## movie.csv
# final dataset
# movieCd = index
# salesAcc(target)
# scrnAcc(target)
# audiAcc(target)
# showAcc(target)
# showTm
# rating
# type
# genres(multi)
#   genreNm
# directors(multi)
#   directorNm
# actors(multi)
#   actorNm
# showTypes(multi)
#   showTypeNm
# companys(multi)
#   companyCd
# watchGrade
# nations(multi)
#   nationNm

# with open('movie_detail.json', 'r') as f :
#     movie_detail = json.load(f)
# 
#
movie_rating_list = pd.read_csv('movie_rating_list.csv')


movie_csv_header = ['movieCd',       # 0
                    'showTm',        # 1
                    'rating',        # 2
                    'type',          # 3
                    'genre_1',       # 4
                    'genre_2',       # 5
                    'genre_3',       # 6
                    'director_1',    # 7
                    'director_2',    # 8
                    'actor_1',       # 9
                    'actor_2',       # 10
                    'actor_3',       # 11
                    'actor_4',       # 12
                    'actor_5',       # 13
                    'showType_1',    # 14
                    'showType_2',    # 15
                    'showType_3',    # 16
                    'company_1',     # 17
                    'company_2',     # 18
                    'company_3',     # 19
                    'company_4',     # 20
                    'grade',         # 21
                    'nation_1',      # 22
                    'nation_2',      # 23
                    'multi',         # 24
                    'domestic',      # 25
                    'salesAcc',      # 26
                    'scrnAcc',       # 27
                    'audiAcc',       # 28
                    'showAcc'        # 29
                   ]

movie_result = []

for i in movie_detail :
    md = movie_detail[i]
    tmp_list = []
    
    tmp_list.append(md['movieCd'])
    
    tmp_list.append(md['showTm'])
    
    k = movie_rating_list[movie_rating_list['movieCd'] == md['movieCd']]
    try :
        tmp_list.append(k.iloc[0].rating)
    except :
        print(i)
    
    tmp_list.append(movie_data_table['type'][md['typeNm']])
    
    cnt = 0
    for j in md['genres'] :
        tmp_list.append(movie_data_table['genre'][j['genreNm']])
        cnt = cnt + 1
        if cnt == 3 :
            break
    for j in range(cnt,3) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['directors'] :
        tmp_list.append(movie_data_table['director'][j['peopleNm']])
        cnt = cnt + 1
        if cnt == 2 :
            break
    for j in range(cnt, 2) :
        tmp_list.append(float('nan'))
        
    cnt = 0
    for j in md['actors'] :
        tmp_list.append(movie_data_table['actor'][j['peopleNm']])
        cnt = cnt + 1
        if cnt == 5 :
            break
    for j in range(cnt, 5) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['showTypes'] :
        tmp_list.append(movie_data_table['showType'][j['showTypeGroupNm']])
        cnt = cnt + 1
        if cnt == 3 :
            break
    for j in range(cnt, 3) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['companys'] :
        tmp_list.append(j['companyCd'])
        cnt = cnt + 1
        if cnt == 4 :
            break
    for j in range(cnt, 4) :
        tmp_list.append(float('nan'))
        
    try :
        tmp_list.append(movie_data_table['grade'][md['audits'][0]['watchGradeNm']])
    except :
        tmp_list.append(float('nan'))
        
    cnt = 0
    for j in md['nations'] :
        tmp_list.append(movie_data_table['nation'][j['nationNm']])
        cnt = cnt + 1
        if cnt == 2 :
            break
    for j in range(cnt, 2) :
        tmp_list.append(float('nan'))
    
    try :
        k = movie_data[movie_data['movieCd'] == md['movieCd']]
        k = k.iloc[0]
        # print(k)
        if(k['multi'] == 'Y') :
            tmp_list.append(1)
        else :
            tmp_list.append(0)
            
        if(k['nation'] == 'F') :
            tmp_list.append(0)
        else :
            tmp_list.append(1)

        tmp_list.append(k['salesAcc'])

        #screen
        tmp_list.append(k['screen'])

        #audiAcc
        tmp_list.append(k['audienceAcc'])

        #show
        tmp_list.append(k['show'])
    except :
        print('..')
    # print(tmp_list)
    movie_result.append(tmp_list)
    

movie = pd.DataFrame(movie_result, columns = movie_csv_header)
movie.to_csv('movie.csv')

#tmp_list.to_csv(tmp_list, movie_csv_header)

20122066
..
20124356
..
20136450
..


### movie data table

revise movie_data_table

1. grade 

In [287]:
with open('movie_data_table.json', 'r') as f :
    movie_data_table = json.load(f)

In [288]:
grade = movie_data_table['grade']
grade

{'12세관람가': '0',
 '전체관람가': '1',
 '12세이상관람가': '2',
 '15세관람가': '3',
 '12세 미만인 자는 관람할 수 없는 등급': '4',
 '': '5',
 '청소년관람불가': '6',
 '18세관람가': '7',
 '연소자관람불가': '8',
 '15세이상관람가': '9',
 '연소자관람가': '10',
 '고등학생이상관람가': '11',
 '중학생이상관람가': '12',
 '국민학생관람불가': '13',
 '미성년자관람불가': '14',
 '15세 미만인 자는 관람할 수 없는 등급 ': '15',
 '모든 관람객이 관람할 수 있는 등급': '16',
 '18세 미만인 자는 관람할 수 없는 등급': '17',
 '미정': '18',
 '기타': '19'}

In [344]:
import math

In [378]:
value_list = []
for v in grade.values() :
    value_list = v
    
'''
전체 1 10 16
12세 0 2 4 12 13
15세 3 9 11 15
18세 6 7 8 14 17
기타 5 18 19
'''

gra = [['0', '2', '4', '12', '13'], 
       ['1', '10', '16'],
       ['3', '9', '11', '15'],
       ['6', '7', '8', '11', '17']]

grad = ['0',
        '1',
        '0',
        '2',
        '1',
        '4',
        '3',
        '3',
        '3',
        '2',
        '0',
        '2',
        '1',
        '1',
        '3',
        '2',
        '0',
        '3',
        '4',
        '4']

mm = pd.read_csv('movie.csv')
ml = mm.values.tolist()

print(len(gra))

for mt in range(len(ml)) :
    k = ml[mt][21]
    if (math.isnan(k)) :
        ml[mt][21] = 4
    else :
        ml[mt][21] = grad[int(k)]
    
    # print(mm.iloc[mt]['grade'])
    

mldf = pd.DataFrame(ml, columns = mm.columns)

4


In [380]:
mldf.to_csv('movie.csv', index = False)

In [381]:
movie_data_table

{'actor': {'버스터 키튼': '5230000',
  '마리온 맥': '22700',
  '게리 쿠퍼': '116000',
  '마를렌 디트리히': '68200',
  '폴 무니': '82800',
  '로버트 도냇': '631',
  '매들린 캐롤': '20100',
  '찰스 로튼': '16700',
  '클라크 게이블': '84800',
  '프랑코트 톤': '129000',
  '로버트 테일러': '436000',
  '그레타 가르보': '4240000',
  '폴 로버슨': '61300',
  '세드릭 하드윅': '1560',
  '로랜드 영': '39100',
  '자넷 게이노': '22600',
  '프레드릭 마치': '137000',
  '캐리 그랜트': '70500',
  '캐서린 헵번': '39500',
  '에롤 플린': '4190',
  '올리비아 드 하빌랜드': '7730',
  '진 아서': '1600000',
  '라이오넬 배리모어': '3110000',
  '주디 갈란드': '13100',
  '헨리 폰다': '39500',
  '앨리스 브래디': '33000000',
  '로렌스 올리비에': '45500',
  '메를 오베론': '13000',
  '제임스 스튜어트': '375000',
  '그리어 가슨': '2920',
  '오손 웰즈': '9710000',
  '도로시 코밍고어': '2840',
  '조셉 코튼': '343000',
  '바바라 스탠윅': '6870',
  '험프리 보가트': '30600',
  '잉그리드 버그만': '92700',
  '폴 헌레이드': '17500',
  '제임스 카그니': '5590',
  '조안 레슬리': '11300',
  '랜돌프 스콧': '8280',
  '존 웨인': '454000',
  '캐롤 롬바드': '2910',
  '잭 베니': '855000',
  '제니퍼 존스': '187000',
  '찰스 빅포드': '118000',
  '브루스 버네트': '14800',
  

### movie_data_table_nonmulti_kr.json

Scaling



#### 1. Scaling

In [41]:
with open('movie_data_table_nonmulti_kr.json', 'r') as f :
    mdtnk = json.load(f)
    
mdtnk_a = mdtnk['actor']

In [52]:
import numpy as np

In [68]:
awareness_actor_raw = []
for name in mdtnk_a :
    k=mdtnk_a[name]
    try :
        if(mdtnk_a[name][-1]=='개') :
            k = mdtnk_a[name][0:-1]
    except :
        k = mdtnk_a[name]
    awareness_actor_raw.append(k)
    
aar_np = np.asarray(awareness_actor_raw)

In [50]:
from sklearn import preprocessing

In [89]:
aar_np_scaled = preprocessing.minmax_scale(aar_np)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [90]:
mdtnk_a_scaled = pd.DataFrame(np.hstack([aar_np_scaled]), columns=["actor"])

In [91]:
mdtnk_scaled = mdtnk

cnt = 0
for name in mdtnk_scaled['actor'] :
    mdtnk_scaled['actor'][name] = mdtnk_a_scaled.iloc[cnt,0]
    cnt = cnt + 1
    
mdtnk_scaled['actor'].items()

dict_items([('유계선', 0.0001927409261576971), ('전창근', 0.000392991239048811), ('송억', 2.2152690863579473e-06), ('이향', 0.032665832290362955), ('윤인자', 0.0001839799749687109), ('이민자', 0.015269086357947435), ('황해남', 1.1964956195244055e-05), ('지학자', 0.01902377972465582), ('박노식', 0.0003566958698372966), ('황남', 0.0011201501877346684), ('허장강', 0.0003216520650813517), ('최강', 0.11151439299123905), ('김지미', 0.0010262828535669588), ('김정훈', 0.008160200250312891), ('김승호', 0.0011376720901126408), ('최남현', 8.172715894868586e-05), ('최은희', 0.001314142678347935), ('김진규', 0.0008448060075093867), ('장민호', 0.0006195244055068836), ('조미령', 0.0006433041301627033), ('이수련', 0.00048811013767209013), ('황정순', 0.00028410513141426786), ('황해', 0.002653316645807259), ('윤일봉', 0.00027033792240300375), ('문정숙', 0.000327909887359199), ('김혜정', 0.0018648310387984982), ('윤정란', 2.8660826032540678e-05), ('도금봉', 0.0001276595744680851), ('엄앵란', 0.002027534418022528), ('신영균', 0.0012027534418022527), ('김희갑', 0.00031289111389236547), ('고은아'

In [92]:
with open("movie_data_table_nonmulti_kr_scaled.json", 'w', encoding = "utf-8") as output :
    json.dump(mdtnk_scaled, output)

## movie_nonmulti_kr_scaled.csv


In [99]:
mdtnk_scaled

{'actor': {'유계선': 0.0001927409261576971,
  '전창근': 0.000392991239048811,
  '송억': 2.2152690863579473e-06,
  '이향': 0.032665832290362955,
  '윤인자': 0.0001839799749687109,
  '이민자': 0.015269086357947435,
  '황해남': 1.1964956195244055e-05,
  '지학자': 0.01902377972465582,
  '박노식': 0.0003566958698372966,
  '황남': 0.0011201501877346684,
  '허장강': 0.0003216520650813517,
  '최강': 0.11151439299123905,
  '김지미': 0.0010262828535669588,
  '김정훈': 0.008160200250312891,
  '김승호': 0.0011376720901126408,
  '최남현': 8.172715894868586e-05,
  '최은희': 0.001314142678347935,
  '김진규': 0.0008448060075093867,
  '장민호': 0.0006195244055068836,
  '조미령': 0.0006433041301627033,
  '이수련': 0.00048811013767209013,
  '황정순': 0.00028410513141426786,
  '황해': 0.002653316645807259,
  '윤일봉': 0.00027033792240300375,
  '문정숙': 0.000327909887359199,
  '김혜정': 0.0018648310387984982,
  '윤정란': 2.8660826032540678e-05,
  '도금봉': 0.0001276595744680851,
  '엄앵란': 0.002027534418022528,
  '신영균': 0.0012027534418022527,
  '김희갑': 0.00031289111389236547,
  '고은아': 

In [102]:
movie_rating_list = pd.read_csv('movie_rating_list.csv', 'r')

In [104]:
## movie_nonmulti_kr_scaled.csv
# final dataset
# movieCd = index
# salesAcc(target)
# scrnAcc(target)
# audiAcc(target)
# showAcc(target)
# showTm
# rating
# type
# genres(multi)
#   genreNm
# directors(multi)
#   directorNm
# actors(multi)
#   actorNm
# showTypes(multi)
#   showTypeNm
# companys(multi)
#   companyCd
# watchGrade
# nations(multi)
#   nationNm

with open('movie_detail.json', 'r') as f :
    movie_detail = json.load(f)
# 
#
movie_rating_list = pd.read_csv('movie_rating_list.csv')


movie_csv_header = ['movieCd',       # 0
                    'showTm',        # 1
                    'rating',        # 2
                    'type',          # 3
                    'genre_1',       # 4
                    'genre_2',       # 5
                    'genre_3',       # 6
                    'director_1',    # 7
                    'director_2',    # 8
                    'actor_1',       # 9
                    'actor_2',       # 10
                    'actor_3',       # 11
                    'actor_4',       # 12
                    'actor_5',       # 13
                    'showType_1',    # 14
                    'showType_2',    # 15
                    'showType_3',    # 16
                    'company_1',     # 17
                    'company_2',     # 18
                    'company_3',     # 19
                    'company_4',     # 20
                    'grade',         # 21
                    'nation_1',      # 22
                    'nation_2',      # 23
                    'multi',         # 24
                    'domestic',      # 25
                    'salesAcc',      # 26
                    'scrnAcc',       # 27
                    'audiAcc',       # 28
                    'showAcc'        # 29
                   ]
def movie_nonmulti_kr_scaled_csv(movie_detail, movie_data, movie_rating_list, mdtnk) :
    movie_result = []

    for i in movie_detail :
        mde = movie_detail[i]
        #print(mda['multi'].iloc[0])
        try :
            mda = movie_data[movie_data['movieCd'] == mde['movieCd']]
            if (mda['multi'].iloc[0] == 'N') & (mda['nation'].iloc[0] == 'K') :
                tmp_list = []

                tmp_list.append(mde['movieCd'])

                tmp_list.append(mde['showTm'])

                k = movie_rating_list[movie_rating_list['movieCd'] == mde['movieCd']]
                try :
                    tmp_list.append(k.iloc[0].rating)
                except :
                    print(i)

                tmp_list.append(mdtnk['type'][mde['typeNm']])

                cnt = 0
                for j in mde['genres'] :
                    tmp_list.append(mdtnk['genre'][j['genreNm']])
                    cnt = cnt + 1
                    if cnt == 3 :
                        break
                for j in range(cnt,3) :
                    tmp_list.append(float('nan'))

                cnt = 0
                for j in mde['directors'] :
                    tmp_list.append(mdtnk['director'][j['peopleNm']])
                    cnt = cnt + 1
                    if cnt == 2 :
                        break
                for j in range(cnt, 2) :
                    tmp_list.append(float('nan'))

                cnt = 0
                for j in mde['actors'] :
                    tmp_list.append(mdtnk['actor'][j['peopleNm']])
                    cnt = cnt + 1
                    if cnt == 5 :
                        break
                for j in range(cnt, 5) :
                    tmp_list.append(float('nan'))

                cnt = 0
                for j in mde['showTypes'] :
                    tmp_list.append(mdtnk['showType'][j['showTypeGroupNm']])
                    cnt = cnt + 1
                    if cnt == 3 :
                        break
                for j in range(cnt, 3) :
                    tmp_list.append(float('nan'))

                cnt = 0
                for j in mde['companys'] :
                    tmp_list.append(j['companyCd'])
                    cnt = cnt + 1
                    if cnt == 4 :
                        break
                for j in range(cnt, 4) :
                    tmp_list.append(float('nan'))

                try :
                    tmp_list.append(mdtnk['grade'][mde['audits'][0]['watchGradeNm']])
                except :
                    tmp_list.append(float('nan'))

                cnt = 0
                for j in mde['nations'] :
                    tmp_list.append(mdtnk['nation'][j['nationNm']])
                    cnt = cnt + 1
                    if cnt == 2 :
                        break
                for j in range(cnt, 2) :
                    tmp_list.append(float('nan'))

                try :
                    k = mda[mda['movieCd'] == mde['movieCd']]
                    k = k.iloc[0]
                    # print(k)
                    if(k['multi'] == 'Y') :
                        tmp_list.append(1)
                    else :
                        tmp_list.append(0)

                    if(k['nation'] == 'F') :
                        tmp_list.append(0)
                    else :
                        tmp_list.append(1)

                    tmp_list.append(k['salesAcc'])
                    #screen
                    tmp_list.append(k['screen'])

                    #audiAcc
                    tmp_list.append(k['audienceAcc'])

                    #show
                    tmp_list.append(k['show'])
                except :
                    print('..')
                # print(tmp_list)
                movie_result.append(tmp_list)
        except :
            continue

    movie_nonmulti_kr_scaled = pd.DataFrame(movie_result, columns = movie_csv_header)
    movie_nonmulti_kr_scaled.to_csv('movie_nonmulti_kr_scaled.csv', index = False)

    #tmp_list.to_csv(tmp_list, movie_csv_header)
    
movie_nonmulti_kr_scaled_csv(movie_detail, movie_data, movie_rating_list, mdtnk_scaled)

In [112]:
mdtnk_scaled['grade'][movie_detail['20144444']['audits'][0]['watchGradeNm']]

'0'

In [117]:
movie_detail['20144444']

{'movieCd': '20144444',
 'movieNm': '아가씨',
 'movieNmEn': 'The Handmaiden',
 'movieNmOg': '',
 'showTm': '144',
 'prdtYear': '2016',
 'openDt': '20160601',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '스릴러'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '박찬욱', 'peopleNmEn': 'PARK Chan-wook'}],
 'actors': [{'peopleNm': '김민희',
   'peopleNmEn': 'KIM Min-hee',
   'cast': '아가씨',
   'castEn': ''},
  {'peopleNm': '김태리', 'peopleNmEn': 'KIM Tae-ri', 'cast': '하녀', 'castEn': ''},
  {'peopleNm': '하정우', 'peopleNmEn': 'HA Jung-woo', 'cast': '백작', 'castEn': ''},
  {'peopleNm': '조진웅',
   'peopleNmEn': 'CHO Jin-woong',
   'cast': '이모부',
   'castEn': ''},
  {'peopleNm': '김해숙',
   'peopleNmEn': 'KIM Hae-sook',
   'cast': '여집사',
   'castEn': ''},
  {'peopleNm': '문소리', 'peopleNmEn': 'MOON So-ri', 'cast': '이모', 'castEn': ''},
  {'peopleNm': '최병모',
   'peopleNmEn': 'CHOE Byeong-mo',
   'cast': '손님',
   'castEn': ''},
  {'peopleNm': '김인우', 'peopleNmEn': 'K

## company normalization

In [124]:
with open('movie_detail.json', 'r') as f :
    movie_detail = json.load(f)
    
movie = pd.read_csv('movie.csv')

with open('movie_data_table.json', 'r') as f :
    movie_data_table = json.load(f)

## movie_data_table_2.json

1. actor re-normalize
2. company normalize

### 1. actor re-normalize 

In [132]:
movie_detail[movie.movieCd[0]]

{'movieCd': '19278002',
 'movieNm': '제너럴',
 'movieNmEn': 'The General',
 'movieNmOg': '',
 'showTm': '107',
 'prdtYear': '1926',
 'openDt': '',
 'prdtStatNm': '기타',
 'typeNm': '장편',
 'nations': [{'nationNm': '미국'}],
 'genres': [{'genreNm': '코미디'}, {'genreNm': '멜로/로맨스'}, {'genreNm': '전쟁'}],
 'directors': [{'peopleNm': '버스터 키튼', 'peopleNmEn': 'Buster Keaton'},
  {'peopleNm': '클라이드 브루크맨', 'peopleNmEn': 'Clyde Bruckman'}],
 'actors': [{'peopleNm': '버스터 키튼',
   'peopleNmEn': 'Buster Keaton',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '마리온 맥',
   'peopleNmEn': 'Marion Mack',
   'cast': '',
   'castEn': ''}],
 'showTypes': [{'showTypeGroupNm': '필름', 'showTypeNm': '필름'},
  {'showTypeGroupNm': '2D', 'showTypeNm': '디지털'}],
 'companys': [],
 'audits': [],
 'staffs': []}

In [135]:
len(movie_detail[movie.movieCd[0]]['actors'])

2

In [144]:
print(len(movie_detail))

6195


In [152]:
movie_data

,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate
0,19278002,제너럴,857000,2.680000e+05,1,1,177,40.000000,N,F,
1,19288001,잔다르크의 수난,2192000,7.306667e+05,3,18,1086,362.000000,N,F,
2,19308006,모로코,3535000,5.387500e+05,4,19,1642,268.750000,N,F,
3,19328008,스카페이스,4104000,1.026000e+06,4,16,2032,508.000000,Y,F,
4,19338008,뮤지컬 42번가,5201000,7.556000e+05,5,25,2579,374.800000,Y,F,
5,19358005,39 계단,2466000,6.165000e+05,4,22,1228,307.000000,Y,F,
6,19358007,바운티 호의 반란,3689000,9.222500e+05,4,16,1832,458.000000,N,F,
7,19368015,춘희,12791000,5.382667e+05,15,60,6397,271.200000,Y,F,
8,19378007,솔로몬 왕의 보고,12627000,1.145286e+06,7,34,6344,572.285714,N,F,
9,19378023,스타 이즈 본 디 오리지널,7030000,8.787500e+05,8,31,3501,437.625000,Y,F,


In [199]:
def actor_by_audience(movie_data, movie_detail, movie_data_table) :    
    actor_audi = dict()
    for cd in movie_detail :
        for actor in movie_detail[cd]['actors'] :
            actor_audi[actor['peopleNm']] = []
    for cd in movie_detail :
        # print(cd)
        # print(movie_data[movie_data.movieCd == cd].audienceAcc.iloc[0])
        cd_audiences = movie_data[movie_data.movieCd == cd].audienceAcc.iloc[0]
        for actor in movie_detail[cd]['actors'] :
            sl = actor_audi[actor['peopleNm']]
            sl.append({cd : str(cd_audiences)})
            actor_audi[actor['peopleNm']] = sl
    
    return actor_audi

aa = actor_by_audience(movie_data, movie_detail, movie_data_table)
with open('actor_audience_data.json', 'w', encoding = 'utf-8') as output :
    json.dump(aa, output)

In [233]:
def actor_normalize_by_audience(movie_data, movie_data_table, actor_audi) :
    mdt2 = movie_data_table
    for actor in mdt2['actor'] :
        mdt2['actor'][actor] = 0
    
    for actor in actor_audi :
        aa_sum = 0
        aa_avg = 0
        for audi in actor_audi[actor] :
            k = list(audi.values())
            aa_sum = aa_sum + int(k[0])
        aa_avg = aa_sum / len(actor_audi[actor])
        mdt2['actor'][actor] = aa_avg
        
    return mdt2
        
mdt2 = actor_normalize_by_audience(movie_data, movie_data_table, aa)

In [155]:
with open('movie_data_table2.json', 'r') as f :
    mdt2 = json.load(f)

In [158]:
actor_audi = mdt2['actor']

In [162]:
def normalize_actor(actor_audi) :
    actor_list = []
    for actor in actor_audi :
        actor_list.append([actor, actor_audi[actor]])
    return actor_list

al = normalize_actor(actor_audi)

In [163]:
actor_audiences = pd.DataFrame(al, columns = ['actorNm', 'avgAudiences'])

In [165]:
actor_audiences_scaled = actor_audiences
actor_audiences_scaled['avgAudiences'] = preprocessing.scale(actor_audiences_scaled['avgAudiences'])

In [238]:
with open('movie_data_table2.json', 'w', encoding = 'utf-8') as output :
    json.dump(mdt2, output)

### 2. company normalize

category : 제작사, 배급사  
sort by sales

In [3]:
with open('movie_detail.json', 'r') as f :
    movie_detail = json.load(f)
    
# go to [build movie_data] to get movie_data

In [14]:
def get_movie_company_category(movie_detail) :
    company_list = []
    company_header = ['companyCd', 'name', 'category', 'totalSales', 'totalMovie']
    company_detail = []
    for cd in movie_detail :
        for mc in movie_detail[cd]['companys'] :
            company_detail = []
            company_detail.append(mc['companyCd'])
            company_detail.append(mc['companyNm'])
            company_detail.append(mc['companyPartNm'])
            company_detail.append('')
            company_detail.append('')
            
            company_list.append(company_detail)
            

    df = pd.DataFrame(company_list, columns = company_header)
    
    return df


movie_company_category = get_movie_company_category(movie_detail)
movie_company_category = movie_company_category.drop_duplicates()
# print(movie_company_category)

In [15]:
mcc = movie_company_category
# print(mcc[mcc.category == '제작사'].drop_duplicates())

In [266]:
def get_company_with_sales(movie_detail, movie_data, key) :
    company_list = []
    for cd in movie_detail :
        msales = movie_data[movie_data['movieCd'] == cd]['salesAcc'].iloc[0]
        companyCd = ''
        for mc in movie_detail[cd]['companys'] :
            cpn = mc['companyPartNm']
            if (cpn == key) :
                companyCd = mc['companyCd']
                companyNm = mc['companyNm']
                company_list.append([
                    companyCd,
                    companyNm,
                    cd,
                    msales,
                ])
    
    return company_list

# key : '배급사', '제작사'
company_sales = get_company_with_sales(movie_detail, movie_data, '제작사')

In [264]:
cls_s = company_sales

In [265]:
df_cls = pd.DataFrame(cls_s, columns = ['companyCd', 'companyNm', 'movieCd', 'sales'])
cls_group = df_cls.groupby(['companyCd'])
company_sales_supply = pd.DataFrame({'companyNm' : cls_group['companyNm'].min(),
                                     'salesAcc' : cls_group['sales'].sum(),
                                     'avgSales' : cls_group['sales'].mean(),
                                     'number' : cls_group['movieCd'].count()}).reset_index()
company_sales_supply = company_sales_supply.sort_values(by='salesAcc', ascending = False).reset_index(drop = True)
company_sales_supply

,companyCd,companyNm,salesAcc,avgSales,number
0,20110854,씨제이이앤엠(주),2282219040589,1.426387e+10,160
1,20100103,(주)쇼박스,1394533961349,2.324223e+10,60
2,20100932,(주)넥스트엔터테인먼트월드(NEW),1132249765237,1.068160e+10,106
3,20100520,롯데쇼핑㈜롯데엔터테인먼트,1099369367011,8.456687e+09,130
4,20100043,워너브러더스 코리아(주),1093901193860,1.439344e+10,76
5,20100041,이십세기폭스코리아(주),953746982600,1.238632e+10,77
6,20100603,유니버설픽쳐스인터내셔널 코리아(유),722431801234,6.020265e+09,120
7,20161801,월트디즈니컴퍼니코리아 유한책임회사,428605217922,2.040977e+10,21
8,20060452,월트디즈니컴퍼니코리아(주),426234619060,2.367970e+10,18
9,20125636,소니픽쳐스릴리징월트디즈니스튜디오스코리아(주),349053416414,1.125979e+10,31


In [23]:
df_cls

,companyCd,companyNm,movieCd,sales
0,20101168,RKO 라디오 픽쳐스,19380009,4257000
1,20101670,워너브러더스사㈜,19420001,35391500
2,20100481,고려영화(주),19460021,988000
3,20104469,파라마운트 픽쳐스,19508004,4360000
4,20104469,파라마운트 픽쳐스,19518007,420000
5,20103668,워너 브러더스 픽쳐스,19518015,22390500
6,20104469,파라마운트 픽쳐스,19518016,19309000
7,20104352,메트로-골드윈-마이어,19538014,20194000
8,20102771,콜럼비아 픽쳐스 코포레이션,19538016,35101000
9,20104469,파라마운트 픽쳐스,19538019,21489000


In [267]:
cls_m = company_sales

In [347]:
cls_m

[['20101168', 'RKO 라디오 픽쳐스', '19380009', 4257000],
 ['20101670', '워너브러더스사㈜', '19420001', 35391500],
 ['20100481', '고려영화(주)', '19460021', 988000],
 ['20104469', '파라마운트 픽쳐스', '19508004', 4360000],
 ['20104469', '파라마운트 픽쳐스', '19518007', 420000],
 ['20103668', '워너 브러더스 픽쳐스', '19518015', 22390500],
 ['20104469', '파라마운트 픽쳐스', '19518016', 19309000],
 ['20104352', '메트로-골드윈-마이어', '19538014', 20194000],
 ['20102771', '콜럼비아 픽쳐스 코포레이션', '19538016', 35101000],
 ['20104469', '파라마운트 픽쳐스', '19538019', 21489000],
 ['20104469', '파라마운트 픽쳐스', '19548002', 23400000],
 ['20101413', '월트 디즈니 프로덕션', '19548004', 21995000],
 ['20104352', '메트로-골드윈-마이어', '19548009', 11948000],
 ['20101507', '이십세기폭스필름코퍼레이션', '19548012', 2524000],
 ['20104469', '파라마운트 픽쳐스', '19548014', 27149500],
 ['20101507', '이십세기폭스필름코퍼레이션', '19548017', 3807000],
 ['20101345', '토호 컴퍼니', '19548035', 5028000],
 ['20102771', '콜럼비아 픽쳐스 코포레이션', '19558004', 13250000],
 ['20102771', '콜럼비아 픽쳐스 코포레이션', '19558005', 4589000],
 ['20103668', '워너 브러더스 픽쳐스', '195

In [268]:
df_cls = pd.DataFrame(cls_m, columns = ['companyCd', 'companyNm', 'movieCd', 'sales'])
cls_group = df_cls.groupby(['companyCd'])
company_sales_making = pd.DataFrame({'companyNm' : cls_group['companyNm'].min(),
                                     'salesAcc' : cls_group['sales'].sum(),
                                     'avgSales' : cls_group['sales'].mean(),
                                     'number' : cls_group['movieCd'].count()}).reset_index()
company_sales_making = company_sales_making.sort_values(by='salesAcc', ascending = False).reset_index(drop = True)
company_sales_making

,companyCd,companyNm,salesAcc,avgSales,number
0,20110854,씨제이이앤엠(주),531180698579,2.309481e+10,23
1,20100278,(주)제이케이필름,391489880261,4.893624e+10,8
2,20100540,리얼라이즈픽쳐스(주),313556407217,7.838910e+10,4
3,20100103,(주)쇼박스,275143138354,2.751431e+10,10
4,20100374,(주)외유내강,253673433384,3.170918e+10,8
5,20100450,영화사 집,228633932705,3.266199e+10,7
6,20122737,(주)사나이픽처스,227675582779,2.845945e+10,8
7,20160941,(주)덱스터스튜디오,218370838046,1.091854e+11,2
8,20062083,(주)케이퍼필름,192134260281,9.606713e+10,2
9,20186561,(주)필름케이,179355732787,5.978524e+10,3


In [269]:
company_sales_supply.to_csv('company_sales_supply.csv', encoding = 'utf-8', index = False)
company_sales_making.to_csv('company_sales_making.csv', encoding = 'utf-8', index = False)

In [4]:
company_sales_supply = pd.read_csv('company_sales_supply.csv')
company_sales_making = pd.read_csv('company_sales_making.csv')

In [270]:
css = company_sales_supply
csm = company_sales_making

In [271]:
css = css.sort_values(by='salesAcc', ascending = False).reset_index(drop = True)
csm = csm.sort_values(by='avgSales', ascending = False).reset_index(drop = True)

In [272]:
csm.head(20)

,companyCd,companyNm,salesAcc,avgSales,number
0,20160941,(주)덱스터스튜디오,218370838046,1.091854e+11,2
1,20062083,(주)케이퍼필름,192134260281,9.606713e+10,2
2,20100934,(주)씨엘엔터테인먼트,91433282670,9.143328e+10,1
3,20100540,리얼라이즈픽쳐스(주),313556407217,7.838910e+10,4
4,20140512,(주)예인플러스엔터테인먼트,62714788849,6.271479e+10,1
5,20155988,(주)영화사하얼빈,61269259831,6.126926e+10,1
6,20186561,(주)필름케이,179355732787,5.978524e+10,3
7,20160263,(주)우정필름,58166181145,5.816618e+10,1
8,20100386,(주)어나더썬데이,57529427417,5.752943e+10,1
9,20169368,(주)사이드미러,55863192382,5.586319e+10,1


In [33]:
def data_norm(target, key) :
    t_max = target[key].max()
    t_min = target[key].min()
    
    target[key] = (target[key] - t_min) / (t_max - t_min)
    
    return target
'''
css_max = css['sales'].max()
css_min = css['sales'].min()

css['sales'] = (css['sales'] - css_min) / (css_max - css_min)
'''

css = data_norm(css, 'salesAcc')
css = data_norm(css, 'avgSales')

,companyCd,companyNm,sales,avgSales,number
0,20110854,씨제이이앤엠(주),1.000000e+00,2.115192e-01,23
1,20100278,(주)제이케이필름,7.370183e-01,4.481939e-01,8
2,20100540,리얼라이즈픽쳐스(주),5.903008e-01,7.179448e-01,4
3,20100103,(주)쇼박스,5.179841e-01,2.519962e-01,10
4,20100374,(주)외유내강,4.775652e-01,2.904159e-01,8
5,20100450,영화사 집,4.304259e-01,2.991424e-01,7
6,20122737,(주)사나이픽처스,4.286217e-01,2.606525e-01,8
7,20160941,(주)덱스터스튜디오,4.111046e-01,1.000000e+00,2
8,20062083,(주)케이퍼필름,3.617117e-01,8.798531e-01,2
9,20186561,(주)필름케이,3.376548e-01,5.475570e-01,3


In [8]:
import numpy as np
from sklearn import preprocessing

In [273]:
css['sales'] = preprocessing.scale(css['salesAcc'])
css['avgSales'] = preprocessing.scale(css['avgSales'])

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [274]:
csm['sales'] = preprocessing.scale(csm['salesAcc'])
csm['avgSales'] = preprocessing.scale(csm['avgSales'])

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [275]:
css.to_csv('company_sales_supply_scaled.csv')
csm.to_csv('company_sales_making_scaled.csv')

### normalize genre

In [78]:
def normalize_genre(movie_detail, movie_data) :
    genre_list = []
    for cd in movie_detail :
        md = movie_detail[cd]
        msales = movie_data[movie_data['movieCd'] == cd].salesAcc.iloc[0]
        for genre in md['genres'] :
            # print(gerne + str(msales))
            genre_list.append([genre['genreNm'], cd, msales])
            
    return genre_list

gl = normalize_genre(movie_detail, movie_data)
gl

[['코미디', '19278002', 857000],
 ['멜로/로맨스', '19278002', 857000],
 ['전쟁', '19278002', 857000],
 ['드라마', '19288001', 2192000],
 ['멜로/로맨스', '19308006', 3535000],
 ['범죄', '19328008', 4104000],
 ['뮤지컬', '19338008', 5201000],
 ['멜로/로맨스', '19338008', 5201000],
 ['드라마', '19338008', 5201000],
 ['미스터리', '19358005', 2466000],
 ['스릴러', '19358005', 2466000],
 ['어드벤처', '19358007', 3689000],
 ['드라마', '19358007', 3689000],
 ['멜로/로맨스', '19368015', 12791000],
 ['드라마', '19368015', 12791000],
 ['액션', '19378007', 12627000],
 ['어드벤처', '19378007', 12627000],
 ['드라마', '19378023', 7030000],
 ['가족', '19380009', 4257000],
 ['멜로/로맨스', '19380009', 4257000],
 ['코미디', '19380009', 4257000],
 ['액션', '19388001', 3255000],
 ['어드벤처', '19388001', 3255000],
 ['멜로/로맨스', '19388001', 3255000],
 ['코미디', '19388005', 4184000],
 ['멜로/로맨스', '19388005', 4184000],
 ['드라마', '19388012', 2700000],
 ['가족', '19398001', 12517000],
 ['판타지', '19398001', 12517000],
 ['어드벤처', '19398001', 12517000],
 ['뮤지컬', '19398001', 12517000],
 ['드라마', '1939

In [79]:
df_gl = pd.DataFrame(gl, columns = ['genreNm', 'movieCd', 'sales'])
df_gl

,genreNm,movieCd,sales
0,코미디,19278002,857000
1,멜로/로맨스,19278002,857000
2,전쟁,19278002,857000
3,드라마,19288001,2192000
4,멜로/로맨스,19308006,3535000
5,범죄,19328008,4104000
6,뮤지컬,19338008,5201000
7,멜로/로맨스,19338008,5201000
8,드라마,19338008,5201000
9,미스터리,19358005,2466000


In [84]:
gl_group = df_gl.groupby(['genreNm'])
genre_sales = pd.DataFrame({'salesAcc' : gl_group['sales'].sum(),
                            'avgSales' : gl_group['sales'].mean(),
                            'number' : gl_group['movieCd'].count()}).reset_index()
genre_sales = genre_sales.sort_values(by = 'avgSales', ascending = False).reset_index(drop = True)
genre_sales

,genreNm,salesAcc,avgSales,number
0,SF,2001551100939,9.622842e+09,208
1,사극,718211470334,7.722704e+09,93
2,액션,5001526589783,6.347115e+09,788
3,어드벤처,2565287652915,5.924452e+09,433
4,판타지,1410479579363,5.019500e+09,281
5,범죄,1454763040311,4.965062e+09,293
6,스릴러,1735122889317,2.794079e+09,621
7,미스터리,483535797564,2.671468e+09,181
8,뮤지컬,228198087310,2.480414e+09,92
9,코미디,1752011133478,2.386936e+09,734


In [85]:
genre_sales_scaled = genre_sales
genre_sales_scaled['salesAcc'] = preprocessing.scale(genre_sales['salesAcc'])
genre_sales_scaled['avgSales'] = preprocessing.scale(genre_sales['avgSales'])
genre_sales_scaled

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


,genreNm,salesAcc,avgSales,number
0,SF,0.575417,2.528785,208
1,사극,-0.339617,1.822388,93
2,액션,2.714431,1.310998,788
3,어드벤처,0.977367,1.153869,433
4,판타지,0.153977,0.817444,281
5,범죄,0.185552,0.797205,293
6,스릴러,0.385451,-0.009880,621
7,미스터리,-0.506944,-0.055462,181
8,뮤지컬,-0.689002,-0.126489,92
9,코미디,0.397493,-0.161240,734


In [150]:
genre_sales_scaled.to_csv('genre_sales_scaled.csv')

### normalize director

In [87]:
def normalize_director(movie_detail, movie_data) :
    director_list = []
    for cd in movie_detail :
        md = movie_detail[cd]
        msales = movie_data[movie_data['movieCd'] == cd].salesAcc.iloc[0]
        for director in md['directors'] :
            director_list.append([director['peopleNm'], cd, msales])
        
    return director_list

director_list = normalize_director(movie_detail, movie_data)

In [91]:
df_dl = pd.DataFrame(director_list, columns = ['directorNm', 'movieCd', 'sales'])

In [93]:
dl_group = df_dl.groupby(['directorNm'])
dl_sales = pd.DataFrame({'salesAcc' : dl_group['sales'].sum(),
                         'avgSales' : dl_group['sales'].mean(),
                         'number' : dl_group['movieCd'].count()}).reset_index()
dl_sales = dl_sales.sort_values(by = 'avgSales', ascending = False).reset_index(drop = True)
dl_sales

,directorNm,salesAcc,avgSales,number
0,윤제균,191965083730,9.598254e+10,2
1,이환경,91433282670,9.143328e+10,1
2,조스 웨던,88582586366,8.858259e+10,1
3,제니퍼 리,82458547500,8.245855e+10,1
4,크리스 벅,82458547500,8.245855e+10,1
5,이일형,77319791264,7.731979e+10,1
6,최동훈,228315726981,7.610524e+10,3
7,김용화,227708814046,7.590294e+10,3
8,안소니 루소,204299091981,6.809970e+10,3
9,조 루소,204299091981,6.809970e+10,3


In [102]:
director_sales_scaled = dl_sales
director_sales_scaled['salesAcc'] = preprocessing.scale(director_sales_scaled['salesAcc'])
director_sales_scaled['avgSales'] = preprocessing.scale(director_sales_scaled['avgSales'])
director_sales_scaled

,directorNm,salesAcc,avgSales,number
0,윤제균,13.053851,14.650529,2
1,이환경,6.106369,13.944745,1
2,조스 웨던,5.909365,13.502481,1
3,제니퍼 리,5.486149,12.552381,1
4,크리스 벅,5.486149,12.552381,1
5,이일형,5.131024,11.755141,1
6,최동훈,15.565946,11.566712,3
7,김용화,15.524004,11.535326,3
8,안소니 루소,13.906221,10.324711,3
9,조 루소,13.906221,10.324711,3


In [151]:
director_sales_scaled.to_csv('director_sales_scaled.csv')

### normalize nation

In [96]:
def normalize_nation(movie_detail, movie_data) :
    nation_list = []
    for cd in movie_detail :
        md = movie_detail[cd]
        msales = movie_data[movie_data['movieCd'] == cd].salesAcc.iloc[0]
        for nation in md['nations'] :
            nation_list.append([nation['nationNm'], cd, msales])
    
    return nation_list

nation_list = normalize_nation(movie_detail, movie_data)

In [97]:
df_nl = pd.DataFrame(nation_list, columns = ['nationNm', 'movieCd', 'sales'])

In [100]:
nl_group = df_nl.groupby(['nationNm'])
nl_sales = pd.DataFrame({'salesAcc' : nl_group['sales'].sum(),
                         'avgSales' : nl_group['sales'].mean(),
                         'number' : nl_group['movieCd'].count()}).reset_index()
nl_sales = nl_sales.sort_values(by = 'avgSales', ascending = False).reset_index(drop = True)
nl_sales

,nationNm,salesAcc,avgSales,number
0,뉴질랜드,78285050381,6.523754e+09,12
1,체코,68262735150,6.205703e+09,11
2,호주,121870104684,3.293787e+09,37
3,미국,5190651032106,3.143944e+09,1651
4,필리핀,2991824739,2.991825e+09,1
5,한국,6499526661206,2.239671e+09,2902
6,태국,26702221341,1.780148e+09,15
7,영국,554253468967,1.753967e+09,316
8,홍콩,105884501100,1.393217e+09,76
9,핀란드,8714581844,1.089323e+09,8


In [103]:
nation_sales_scaled = nl_sales
nation_sales_scaled['salesAcc'] = preprocessing.scale(nation_sales_scaled['salesAcc'])
nation_sales_scaled['avgSales'] = preprocessing.scale(nation_sales_scaled['avgSales'])
nation_sales_scaled

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


,nationNm,salesAcc,avgSales,number
0,뉴질랜드,-0.117241,4.809836,12
1,체코,-0.127482,4.550781,11
2,호주,-0.072703,2.179002,37
3,미국,5.106864,2.056954,1651
4,필리핀,-0.194180,1.933051,1
5,한국,6.444347,1.320417,2902
6,태국,-0.169951,0.946131,15
7,영국,0.369131,0.924806,316
8,홍콩,-0.089038,0.630973,76
9,핀란드,-0.188332,0.383449,8


In [152]:
nation_sales_scaled.to_csv('nation_sales_scaled.csv')

### grade normalize

In [125]:
def normalize_grade(movie_detail, movie) :
    grade_list = []
    for cd in movie['movieCd'] :
        grade = movie[movie['movieCd'] == cd]['grade'].iloc[0]
        msales = movie[movie['movieCd'] == cd]['salesAcc'].iloc[0]
        grade_list.append([grade, cd, msales])
    
    return grade_list

grade_list = normalize_grade(movie_detail, movie)

In [126]:
df_gl = pd.DataFrame(grade_list, columns = ['grade', 'movieCd', 'sales'])
gl_group = df_gl.groupby(['grade'])
grade_sales = pd.DataFrame({'salesAcc' : gl_group['sales'].sum(),
                            'avgSales' : gl_group['sales'].mean(),
                            'number' : gl_group['movieCd'].count()}).reset_index()
grade_sales = grade_sales.sort_values(by = 'avgSales', ascending = False).reset_index(drop = True)

In [153]:
grade_sales_scaled = grade_sales
grade_sales_scaled['salesAcc'] = preprocessing.scale(grade_sales_scaled['salesAcc'])
grade_sales_scaled['avgSales'] = preprocessing.scale(grade_sales_scaled['avgSales'])

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [154]:
grade_sales_scaled.to_csv('grade_sales_scaled.csv')

In [142]:
with open('movie_data_table2.json', 'r') as f :
    movie_data_table = json.load(f)

In [144]:
movie_data_table['director']

{'버스터 키튼': '0',
 '클라이드 브루크맨': '1',
 '칼 테오도르 드레이어': '2',
 '조세프 본 스텐버그': '3',
 '하워드 혹스': '4',
 '리차드 로손': '5',
 '로이드 베이컨': '6',
 '알프레드 히치콕': '7',
 '프랭크 로이드': '8',
 '조지 쿠커': '9',
 '로버트 스티븐슨': '10',
 '윌리엄 A. 웰만': '11',
 '마이클 커티즈': '12',
 '윌리엄 케일리': '13',
 '프랭크 카프라': '14',
 '마르셀 카르네': '15',
 '빅터 플레밍': '16',
 '존 포드': '17',
 '윌리엄 와일러': '18',
 '로버트 Z. 레너드': '19',
 '오손 웰즈': '20',
 '레이 앤라이트': '21',
 '머빈 르로이': '22',
 '에른스트 루비치': '23',
 '헨리 킹': '24',
 '졸탄 코다': '25',
 '아서 루빈': '26',
 '장 들라누와': '27',
 '레오 맥커레이': '28',
 '로렌스 올리비에': '29',
 '마이클 파웰': '30',
 '에메릭 프레스버거': '31',
 '데이빗 린': '32',
 '빌리 와일더': '33',
 '최인규': '34',
 '테이 가넷': '35',
 '로베르토 로셀리니': '36',
 '진 네글레스코': '37',
 '델머 데이브즈': '38',
 '제임스 에드워드 그랜트': '39',
 '줄리앙 뒤비비에르': '40',
 '알란 드완': '41',
 '라울 월쉬': '42',
 '로버트 로즌': '43',
 '빈센트 미넬리': '44',
 '클라이드 제로니미': '45',
 '해밀튼 러스크': '46',
 '윌프레드 잭슨': '47',
 '존 휴스턴': '48',
 '엘리아 카잔': '49',
 '조지 스티븐스': '50',
 '윌리엄 디터리': '51',
 '진 켈리': '52',
 '스탠리 도넌': '53',
 '로버트 시오드막': '54',
 '안소니 만': '55',
 '나단 주런': '56'

In [139]:

grad = ['0',
        '1',
        '0',
        '2',
        '1',
        '4',
        '3',
        '3',
        '3',
        '2',
        '0',
        '2',
        '1',
        '1',
        '3',
        '2',
        '0',
        '3',
        '4',
        '4']
print(grad[0])
for g in movie_data_table['grade'] :
    print(g + movie_data_table['grade'][g])
    movie_data_table['grade'][g] = grad[int(movie_data_table['grade'][g])]
    

0
12세관람가0
전체관람가1
12세이상관람가2
15세관람가3
12세 미만인 자는 관람할 수 없는 등급4
5
청소년관람불가6
18세관람가7
연소자관람불가8
15세이상관람가9
연소자관람가10
고등학생이상관람가11
중학생이상관람가12
국민학생관람불가13
미성년자관람불가14
15세 미만인 자는 관람할 수 없는 등급 15
모든 관람객이 관람할 수 있는 등급16
18세 미만인 자는 관람할 수 없는 등급17
미정18
기타19


{'12세관람가': '0',
 '전체관람가': '1',
 '12세이상관람가': '0',
 '15세관람가': '2',
 '12세 미만인 자는 관람할 수 없는 등급': '1',
 '': '4',
 '청소년관람불가': '3',
 '18세관람가': '3',
 '연소자관람불가': '3',
 '15세이상관람가': '2',
 '연소자관람가': '0',
 '고등학생이상관람가': '2',
 '중학생이상관람가': '1',
 '국민학생관람불가': '1',
 '미성년자관람불가': '3',
 '15세 미만인 자는 관람할 수 없는 등급 ': '2',
 '모든 관람객이 관람할 수 있는 등급': '0',
 '18세 미만인 자는 관람할 수 없는 등급': '3',
 '미정': '4',
 '기타': '4'}

In [141]:
with open('movie_data_table.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_data_table, output)

### movie_data_table_norm

| key | value |
|---|:---:|
| actor | avgAudiences |
| director | avgSales |
| genre | avgSales|
| nation | avgSales |
| grade | avgSales|
| company_supply | salesAcc |
| company_making | avgSales |

In [276]:
def make_movie_data_table_norm(act_s, dir_s, gen_s, nat_s, gra_s, css, csm) :
    mdtn = {'actor' : {},
            'director' : {},
            'genre' : {},
            'nation' : {},
            'grade' : {},
            'company_supply' : {},
            'company_making' : {}}
    for i in range(len(act_s)) :
        mdtn['actor'][act_s.actorNm.iloc[i]] = act_s.avgAudiences.iloc[i]
    for i in range(len(dir_s)) :
        mdtn['director'][dir_s.directorNm.iloc[i]] = dir_s.avgSales.iloc[i]
    for i in range(len(gen_s)) :
        mdtn['genre'][gen_s.genreNm.iloc[i]] = gen_s.avgSales.iloc[i]
    for i in range(len(nat_s)) :
        mdtn['nation'][nat_s.nationNm.iloc[i]] = nat_s.avgSales.iloc[i]
    for i in range(len(gra_s)) :
        mdtn['grade'][gra_s.grade.iloc[i]] = gra_s.avgSales.iloc[i]
    for i in range(len(css)) :
        mdtn['company_supply'][css.companyNm.iloc[i]] = css.sales.iloc[i]
    for i in range(len(csm)) :
        mdtn['company_making'][csm.companyNm.iloc[i]] = csm.avgSales.iloc[i]
    return mdtn

movie_data_table_norm = make_movie_data_table_norm(actor_audiences_scaled,
                                                   director_sales_scaled,
                                                   genre_sales_scaled,
                                                   nation_sales_scaled,
                                                   grade_sales_scaled,
                                                   css,
                                                   csm)

### movie_nonmulti_scaled.csv
  
<br>
scaled value : director, actor, company_supply, company_making, genre, grade, nation

<br> 
  
| number | columns |
|:---:|:---:|
| 1 | movieCd |
| 2 | showTm |
| 3 | rating |
| 4 | director_cnt |
| 5 | director_1 |
| 6 | director_2 |
| 7 | actor_cnt |
| 8 | actor_1 |
| 9 | actor_2 |
| 10 | actor_3 |
| 11 | actor_4 |
| 12 | actor_5 |
| 13 | company_supply |
| 14 | company_making |
| 15 | genre_cnt |
| 16 | genre_1 |
| 17 | genre_2 |
| 18 | genre_3 |
| 19 | grade |
| 20 | nation_cnt |
| 21 | nation_1 |
| 22 | nation_2 |
| 23 | salesAcc |
| 24 | audiAcc |
| 25 | director |
| 26 | actor |
| 27 | genre |
| 28 | nation |

In [311]:
def make_movie_nonmulti_scaled_csv(mdtn, mdt, movie_detail, movie) :
    mns = []
    mns_header = ['movieCd', 'showTm', 'rating', 'director_cnt', 'director_1',
                  'director_2', 'actor_cnt', 'actor_1', 'actor_2', 'actor_3',
                  'actor_4', 'actor_5', 'company_supply', 'company_making', 'genre_cnt',
                  'genre_1', 'genre_2', 'genre_3', 'grade', 'nation_cnt',
                  'nation_1', 'nation_2', 'salesAcc', 'audiAcc',
                  'director', 'actor', 'genre', 'nation']
    # ccc = 0
    for cd in movie_detail :
        md = movie_detail[cd]
        tmp_list = []
        mov = movie[movie['movieCd'] == cd]

        try :
            if(mov['multi'].iloc[0] == 0) :
                try :
                    tmp_list.append(cd)
                    tmp_list.append(mov.showTm.iloc[0])
                    tmp_list.append(mov.rating.iloc[0])
                    
                    dcnt = 0
                    tmp_list.append(min(len(md['directors']),2))
                    for director in md['directors'] :
                        tmp_list.append(mdtn['director'][director['peopleNm']])
                        dcnt = dcnt + 1
                        if(dcnt == 2) :
                            break
                    for i in range(dcnt, 2) :
                        tmp_list.append(float('nan'))
                        
                    acnt = 0
                    tmp_list.append(min(len(md['actors']),5))
                    for actor in md['actors'] :
                        tmp_list.append(mdtn['actor'][actor['peopleNm']])
                        acnt = acnt + 1
                        if(acnt == 5) :
                            break
                    for i in range(acnt, 5) :
                        tmp_list.append(float('nan'))
                        
                    cs_cnt = 0
                    cm_cnt = 0
                    cs_sum = 0
                    cm_sum = 0
                    for company in md['companys'] :
                        #print(company)
                        if(company['companyPartNm'] == '배급사') :
                            #print(cd + ' 배급사')
                            #print(mdtn['company_supply'][company['companyNm']])
                            cs_cnt = cs_cnt + 1
                            cs_sum = cs_sum + mdtn['company_supply'][company['companyNm']]
                        elif(company['companyPartNm'] == '제작사') :
                            #print(cd + ' 제작사')
                            #print(mdtn['company_making'][company['companyNm']])
                            cm_cnt = cm_cnt + 1
                            cm_sum = cm_sum + mdtn['company_making'][company['companyNm']]
                    if(cs_cnt == 0) :
                        tmp_list.append('')
                    else :
                        tmp_list.append(cs_sum / cs_cnt)
                    if(cm_cnt == 0) :
                        tmp_list.append('')
                    else :
                        tmp_list.append(cm_sum / cm_cnt)
                    
                    gcnt = 0
                    tmp_list.append(min(len(md['genres']),3))
                    for genre in md['genres'] :
                        tmp_list.append(mdtn['genre'][genre['genreNm']])
                        gcnt = gcnt + 1
                        if(gcnt == 2) :
                            break
                    for i in range(gcnt, 3) :
                        tmp_list.append(float('nan'))
                        
                    tmp_list.append(mdtn['grade'][mov['grade'].iloc[0]])
                    
                    ncnt = 0
                    tmp_list.append(min(len(md['nations']),2))
                    # if(len(md['nations'])>2) :
                    #    ccc = ccc + 1
                    for nation in md['nations'] :
                        tmp_list.append(mdtn['nation'][nation['nationNm']])
                        ncnt = ncnt + 1
                        if(ncnt == 2) :
                            break
                    for i in range(ncnt, 2) :
                        tmp_list.append(float('nan'))
                        
                    tmp_list.append(mov['salesAcc'].iloc[0])
                    tmp_list.append(mov['audiAcc'].iloc[0])
                    tmp_list.append('')
                    tmp_list.append('')
                    tmp_list.append('')
                    tmp_list.append('')
                except :
                    print('error')
                    break
                else :
                    mns.append(tmp_list)
        except :
            # empty or error dataframe
            print('empty')
            continue
    # print(ccc)
    return (mns, mns_header)

movie_nonmulti_scaled, header = make_movie_nonmulti_scaled_csv(movie_data_table_norm,
                                                               movie_data_table,
                                                               movie_detail,
                                                               movie)

print('done')

empty
empty
empty
done


In [312]:
df_mns = pd.DataFrame(movie_nonmulti_scaled, columns = header)
df_mns.to_csv('movie_nonmulti_scaled.csv')

In [304]:
movie_detail['20182990']

{'movieCd': '20182990',
 'movieNm': '아기곰 보보 구출 대작전',
 'movieNmEn': 'In the forest of HuckyBucky',
 'movieNmOg': 'Dyrene i Hakkebakkeskogen',
 'showTm': '75',
 'prdtYear': '2016',
 'openDt': '20180425',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '노르웨이'}],
 'genres': [{'genreNm': '애니메이션'}],
 'directors': [{'peopleNm': '라스무스 A. 실버르센',
   'peopleNmEn': 'Rasmus A. Sivertsen'}],
 'actors': [],
 'showTypes': [{'showTypeGroupNm': '2D', 'showTypeNm': '디지털더빙'}],
 'companys': [{'companyCd': '20141088',
   'companyNm': '와이드 릴리즈(주)',
   'companyNmEn': 'Wide Release',
   'companyPartNm': '배급사'},
  {'companyCd': '20063233',
   'companyNm': '(주)컴퍼니엘',
   'companyNmEn': '',
   'companyPartNm': '수입사'}],
 'audits': [{'auditNo': '2018-MF00677', 'watchGradeNm': '전체관람가'}],
 'staffs': []}

In [310]:
csm

,companyCd,companyNm,salesAcc,avgSales,number,sales
0,20160941,(주)덱스터스튜디오,218370838046,10.057760,2,6.317703
1,20062083,(주)케이퍼필름,192134260281,8.804873,2,5.525048
2,20100934,(주)씨엘엔터테인먼트,91433282670,8.362309,1,2.482685
3,20100540,리얼라이즈픽쳐스(주),313556407217,7.116500,4,9.193435
4,20140512,(주)예인플러스엔터테인먼트,62714788849,5.619495,1,1.615047
5,20155988,(주)영화사하얼빈,61269259831,5.481437,1,1.571374
6,20186561,(주)필름케이,179355732787,5.339703,3,5.138985
7,20160263,(주)우정필름,58166181145,5.185071,1,1.477625
8,20100386,(주)어나더썬데이,57529427417,5.124257,1,1.458387
9,20169368,(주)사이드미러,55863192382,4.965120,1,1.408047


In [307]:
movie_data_table_norm['company_making']

{'(주)덱스터스튜디오': 10.057759950836417,
 '(주)케이퍼필름': 8.804873031812617,
 '(주)씨엘엔터테인먼트': 8.362308690339253,
 '리얼라이즈픽쳐스(주)': 7.1164995972127585,
 '(주)예인플러스엔터테인먼트': 5.619494778808768,
 '(주)영화사하얼빈': 5.481436803874435,
 '(주)필름케이': 5.3397030983383065,
 '(주)우정필름': 5.1850714219855405,
 '(주)어나더썬데이': 5.124257056362361,
 '(주)사이드미러': 4.965120137053048,
 '(주)알로하픽쳐스': 4.790778841748873,
 '(주)토일렛픽쳐스': 4.790778841748873,
 '(주)제이케이필름': 4.303548012105775,
 '(주)타이거픽쳐스': 4.294934290802795,
 '(주)빅스톤픽쳐스': 4.11011897397308,
 '(주)스튜디오앤뉴': 4.055005667046471,
 '(주)로제타시네마': 3.981643418801009,
 '(주)영화사 월광': 3.8784695273297207,
 '(주)영화사호필름 ': 3.870078795850066,
 '베리굿스튜디오(주)': 3.868589166069115,
 '(주)필름몬스터': 3.8656080674266176,
 '드라마하우스(주)': 3.8656080674266176,
 '레젼데리 픽쳐스': 3.782580486230179,
 '어나더픽쳐스(주)': 3.3100452693469022,
 '아거스필름': 3.1977020551610242,
 '(유)내부자들 문화전문회사': 3.14344812103988,
 '(주)영화사참': 3.085379808747882,
 '(주)씨에이씨엔터테인먼트': 3.0745081991876737,
 '(주)에이치비엔터테인먼트': 3.038213371479674,
 '(주)모팩': 3.019178233394

### 6. input missing rating

In [2]:
movie_rating_list = pd.read_csv('movie_rating_list.csv')
movie = pd.read_csv('movie_nonmulti_scaled.csv')

In [23]:
movie = movie.sort_values(by='salesAcc', ascending=False).reset_index(drop=True)

In [15]:
def impute_missing_rating(row, movieCd, value):
    print(movie_rating_list[movie_rating_list['movieCd'] == str(movieCd)])
    if(movie_rating_list.at[row, 'rating'] == float('nan')):
        movie_rating_list.at[row, 'rating'] = value
    else:
        print('already exist')
    print(movieCd,' : ', movie_rating_list.at[row, 'rating'])
    
impute_missing_rating(5394, 20177478, 8.95)

       movieCd  rating
5394  20177478    8.95
already exist
20177478  :  8.95


In [4]:
with open('movie_detail.json', 'r') as f:
    movie_detail = json.load(f)

In [128]:
for i in range(len(movie)):
#    print(str(movie.at[i, 'rating']))
    if(str(movie.at[i, 'rating']) == str(float('nan'))):
        movieCd = str(movie.at[i, 'movieCd'])
        
        mrv = missing_rating_value(i, str(movieCd))
        try :
            movie.at[i, 'rating'] = mrv
            print(i, movieCd, movie_detail[movieCd]['prdtYear'], movie_detail[movieCd]['movieNm'], movie.at[i, 'salesAcc'], ' imputing : ', mrv)
        except :
            print(i, movieCd, 'error !')

154 20137044 2013 친구 2 21804687513.0  imputing :  nan
203 20168261 2017 원더 우먼 17443897186.0  imputing :  nan
635 20133981 2012 헨젤과 그레텔: 마녀 사냥꾼 3D 2119608500.0  imputing :  nan
721 20158529 2015 조이 1372563200.0  imputing :  nan
838 20080328 2006 미운 오리 새끼의 모험 685318300.0  imputing :  nan
901 20168324 2001 반지의 제왕 : 반지원정대 (확장판) 454796800.0  imputing :  nan
942 20168689 2003 반지의 제왕 : 왕의 귀환 (확장판) 356238500.0  imputing :  nan
952 20168664 2002 반지의 제왕 : 두개의 탑 (확장판) 330379700.0  imputing :  nan
961 20168668 2016 레이디버그 310302500.0  imputing :  nan
992 20162022 2016 대결 251176500.0  imputing :  nan
996 20113553 2012 I AM 245809000.0  imputing :  nan
1082 20173444 2017 링스 124480400.0  imputing :  nan
1175 20150841 2014 카이트 66844000.0  imputing :  nan
1183 19990193 1999 매트릭스 60919600.0  imputing :  nan
1197 20150157 2016 마담 버터플라이 52811000.0  imputing :  nan
1209 20150155 2015 투란도트 49519500.0  imputing :  nan
1212 20188023 2018 BIFAN2018 - 판타스틱 단편 걸작선 3 49200000.0  imputing :  nan
1214 20188062 2018 

1986 20165075 2016 제15회 미쟝센 단편영화제 4만번의 구타 2 2246000.0  imputing :  nan
1990 20186207 2018 MSFF2018 희극지왕 2 2226700.0  imputing :  nan
1993 19730184 1973 엄마와 창녀 2224000.0  imputing :  nan
1997 20177544 2017 MSFF2017 비정성시 3 2211000.0  imputing :  nan
2003 20186188 2018 MSFF2018 비정성시 4 2194100.0  imputing :  nan
2011 20080243 1953 존 휴스턴 회고전-비트 더 데블 2152000.0  imputing :  nan
2012 20183812 2018 SIFF2018-개막식 2149000.0  imputing :  nan
2013 20180329 2018 SESIFF2018 아버지 학교 2148000.0  imputing :  nan
2017 20165068 2016 제15회 미쟝센 단편영화제 사랑에 관한 짧은 필름2 2131500.0  imputing :  nan
2019 20119668 1972 루드비히 2118000.0  imputing :  nan
2032 20177528 2017 MSFF2017 4만번의 구타 2 2069000.0  imputing :  nan
2034 19990053 1999 해피엔드 2059000.0  imputing :  nan
2038 19668028 1966 황야의 무뢰한 2050000.0  imputing :  nan
2039 20183634 2018 SIFF2018-폐막식 2044000.0  imputing :  nan
2047 20121093 1948 죽음의 사막 2017000.0  imputing :  nan
2048 20164582 2016 에코그라운드단편 2010000.0  imputing :  nan
2052 20177525 2017 MSFF2017 희극지왕 1 19845

2474 20151321  제15회 한국 퀴어 영화제 국내단편1 : 관계에서 980000.0  imputing :  nan
2475 20134877 1948 맥베스 980000.0  imputing :  nan
2480 20182930 2018 SPFF 2018 국내단편 3 973000.0  imputing :  nan
2481 20164130 2016 [인디포럼2016] 신작06 그 남자 그 여자의 사정 971000.0  imputing :  nan
2484 20160876 2016 어린 형수 - 무삭제판 965000.0  imputing :  nan
2485 20189424 2017 쾌락 도우미 무삭제 964000.0  imputing :  nan
2490 20184478 2018 DIMA 방송보도제작과 영상제 ‘책갈피’ 2부 - Epilogue 960000.0  imputing :  nan
2492 20177207 2017 DIMA 방송보도제작과 영상제 ‘추억이 오네요’ 1부 960000.0  imputing :  nan
2494 20184477 2018 DIMA 방송보도제작과 영상제 ‘책갈피’ 1부 - Prologue 960000.0  imputing :  nan
2497 20160461 2016 인디애니페스트2016-독립보행1 957000.0  imputing :  nan
2498 20158596 2015 서울독립영화제2015 폐막식 957000.0  imputing :  nan
2504 20184777 2018 2018 중앙영화제 일몰 940000.0  imputing :  nan
2514 20174282 2017 음란 기생 - 무삭제판 931000.0  imputing :  nan
2523 20173501 2017 SIDOF2017_9 와 0 사이/You are my../서른/탈 920000.0  imputing :  nan
2530 20185044 2017 타짜의 목적 무삭제 912000.0  imputing :  nan
2531 20175243

2840 20143720 2014 SIFF2014-경쟁단편 3 648000.0  imputing :  nan
2841 20157586 2015 3월 단편 상상극장-여자, 남자 647000.0  imputing :  nan
2842 20169123 2016 2016 대단한단편영화제-단편 경쟁 섹션 1 647000.0  imputing :  nan
2845 20164649 2016 SIFF2016-선택 3 642400.0  imputing :  nan
2846 20164403 2015 원나잇스탠드 642000.0  imputing :  nan
2848 20168643 2016 2017 한국영화아카데미 33기 졸업영화제 - 개막작 640000.0  imputing :  nan
2849 20158405 2015 서울독립영화제 2015 특별단편 3 638200.0  imputing :  nan
2851 19568004 1956 3인의 난폭한 사람들 635000.0  imputing :  nan
2855 20149128 2014 인디포럼2014 너를 기다리는 동안 624000.0  imputing :  nan
2857 20143725 2014 SIFF2014-경쟁단편 8 622100.0  imputing :  nan
2863 20180874 2018 PDFF2018 장애인미디어운동2 615000.0  imputing :  nan
2865 20180872 2018 PDFF2018 장애인미디어운동1 615000.0  imputing :  nan
2867 20180876 2018 PDFF2018 PDFF경선4 615000.0  imputing :  nan
2868 20180882 2018 PDFF2018 PDFF경선2 615000.0  imputing :  nan
2869 20180875 2018 PDFF2018 PDFF경선3 615000.0  imputing :  nan
2870 20158232 2015 SIDOF2015_찍으라는 영화는 안 찍고 / 아빠가 죽으면 나는 어떡

3119 20176901 2017 또떼미즘 - 박소담 특집 325000.0  imputing :  nan
3123 20171762 2017 제16회 미쟝센 단편영화제 수상작 특별전 단편섹션 3 320000.0  imputing :  nan
3124 20170109 2017 대구단편영화제 배우목격담 김예은 318000.0  imputing :  nan
3126 20185267 2018 [인디포럼2018] 신작전 11 이상한 나라의 하우스 315000.0  imputing :  nan
3127 20170083 2017 대구단편영화제 경쟁1 313000.0  imputing :  nan
3129 20165126 2016 제15회 미쟝센 단편영화제 전년도 수상작 1 312000.0  imputing :  nan
3132 20188690 2018 Diff 2018 경쟁8 308000.0  imputing :  nan
3136 20188707 2018 Diff 2018 경쟁9 304000.0  imputing :  nan
3137 19710022 1971 개구장이 도련님 302000.0  imputing :  nan
3138 20163367 1956 보통사람들 302000.0  imputing :  nan
3139 20141544 2014 2014 GSFF 단편경쟁 4 301000.0  imputing :  nan
3141 20166912 2016 개인교수: 심화학습 (무삭제) 300000.0  imputing :  nan
3144 20167986 2016 젊은 형부 무삭제판 300000.0  imputing :  nan
3145 20088318 2012 배꼽 300000.0  imputing :  5.73
3146 20183162 2017 화끈한 홈서비스 무삭제판 300000.0  imputing :  nan
3150 20189603 2018 음란한 모델 무삭제판 300000.0  imputing :  nan
3160 20150326 2014 인디포럼2015 꿈결의 연

3390 20170791 2017 안양국제청소년영화제 국내경쟁 3 (언더 24) 104000.0  imputing :  nan
3397 20165785 2016 친구 누나 무삭제 100000.0  imputing :  nan
3399 20137070 2013 김조광수 감독 단편 3부작 95682.0  imputing :  nan
3400 20138705 2012 가장자리 94000.0  imputing :  nan
3405 20141582 2014 2014 GSFF-개막식 91000.0  imputing :  nan
3406 20147767 2014 장형윤 감독 특별 기획전 90000.0  imputing :  nan
3407 20167864  글로컬 구애전 단편 3 90000.0  imputing :  nan
3410 20188922 2018 8월 단편 상상극장-장르: 힙합 88000.0  imputing :  nan
3411 20164382 2016 판타지, 마술적 순간 88000.0  imputing :  nan
3416 20156981 2015 2월수요단편극장 - “그 겨울, 바람이 분다” 86000.0  imputing :  nan
3417 20148440 2013 인디피크닉플러스 -  단편2: 기억과 성장 86000.0  imputing :  nan
3418 20149135 2014 인디포럼2014 유토피아 디스토피아 84000.0  imputing :  nan
3419 20161188 2016 FIWOM2016-햇살쏟아지던날/달팽이/십분간휴식 84000.0  imputing :  nan
3422 20137706 2012 패밀리 82656.0  imputing :  nan
3430 20161167 2016 FIWOM2016-여름의 끝/전학생/연지/세희 77000.0  imputing :  nan
3431 20178101 2017 Media art work screening 6월 76000.0  imputing :  nan
3432 19970007 1

In [127]:
def missing_rating_value(i, movieCd):
    ret = float('nan')
    url = 'https://openapi.naver.com/v1/search/movie.json'

    md = movie_detail[movieCd]
    try :
        payload = {
            'query' : md['movieNm'],
            'display' : 100
            #'yearfrom' : int(md['prdtYear']),
            #'yearto' : int(md['prdtYear'])
        }

        headers = {
            'X-Naver-Client-Id' : 'V8LONuGkI0U72r0RY3yo',
            'X-Naver-Client-Secret' : 'VKW2Diwq5d'
        }

        if (movie.at[i, 'rating'] == 0) | (str(movie.at[i, 'rating']) == str(float('nan'))) :
            # print(movie_rating_list.iloc[i, 0], mv['movieNm'] + mv['movieNmEn'])
            try :
                req = requests.get(url, headers = headers, params = payload)
                req.json()['total']
            except :
                ret = float('nan')
            else :
                if(req.json()['total'] == 1) :
                    ret = req.json()['items'][0]['userRating']
                else :
                    flag = False
                    for j in req.json()['items'] :
                        directors = j['director'].split('|')
                        #print(directors)
                        #print(j['title'][3:-4])
                        if (j['title'][3:-4] == md['movieNm']):
                            #print('found 1 :', j['userRating'])
                        #    flag = True
                            # ret = j['userRating']
                           # print(md['directors'])
                            for jj in md['directors']:
                            #    print('found 111', jj['peopleNm'], directors)
                                if (jj['peopleNm'] in directors) :
                                    # movie_rating_list.iloc[i,1]=j['userRating']
                             #       print('!! : ', j['userRating'])
                                    flag = True
                                    ret = j['userRating']
                                    if(flag == True) :
                                        break
                        if(flag == True) :
                            break
                    #if(flag == True) :
                    #    break
        else :
            ret = movie.at[i, 'rating']
    except :
        print('something wrong')
    return ret
                

In [117]:
mrv = missing_rating_value(5, str(20177478))
mrv

'8.95'

In [51]:
movie_detail['20177478']

{'movieCd': '20177478',
 'movieNm': '어벤져스: 인피니티 워',
 'movieNmEn': 'Avengers: Infinity War',
 'movieNmOg': '',
 'showTm': '149',
 'prdtYear': '2018',
 'openDt': '20180425',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '미국'}],
 'genres': [{'genreNm': '액션'},
  {'genreNm': '어드벤처'},
  {'genreNm': '판타지'},
  {'genreNm': 'SF'}],
 'directors': [{'peopleNm': '안소니 루소', 'peopleNmEn': 'Anthony Russo'},
  {'peopleNm': '조 루소', 'peopleNmEn': 'Joe Russo'}],
 'actors': [{'peopleNm': '로버트 다우니 주니어',
   'peopleNmEn': 'Robert Downey Jr.',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '조슈 브롤린',
   'peopleNmEn': 'Josh Brolin',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '마크 러팔로',
   'peopleNmEn': 'Mark Ruffalo',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '톰 히들스턴',
   'peopleNmEn': 'Tom Hiddleston',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '크리스 에반스',
   'peopleNmEn': 'Chris Evans',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '크리스 헴스워스',
   'peopleNmEn': 'Chris Hemsworth

In [130]:
movie.to_csv('movie_nonmulti_scaled_r.csv', index = False)